# Covid-19 Tracking in the US#
## Paul Athmer ##

Covid-19 Tracking:  
This project takes county wide US statistics for the spread of Covid-19 from the below sources and runs  
  estimates on current cases, additional per capita statistics and weekly changes for progress tracking.  
  This data is then made available to be viewed in a spreadsheet that can be downloaded, in time lapse  
  graphs, or code that can be copied to mapchart.net for map visuals.  

Data Sources: https://www.kaggle.com/fireballbyedimyrnmom/us-counties-covid-19-dataset/data  
              https://www.census.gov/data/datasets/time-series/demo/popest/2010s-counties-total.html#par_textimage_739801612  

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; } </style>"))
display(HTML("<style>div.output_scroll { height: 44em; }</style>"))

In [2]:
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import webbrowser
import urllib.request
from tkinter import *
from tkinter import messagebox
from pandastable import Table, TableModel
%matplotlib inline


In [3]:
# loading of previously saved csv data files
popdf = pd.read_csv('popdf.csv', header = 'infer')     # Contains populations of all US counties created from census.gov website csv file
stpopdf = pd.read_csv('stpopdf.csv', header = 'infer')
USpop = sum(stpopdf['Population'].to_list())

df = pd.read_csv('Maindf.csv', header = 'infer')       # Main dataframe of all data at the county level
df = df.set_index('Index')
df = df.astype({'Date': np.datetime64})
StateDF = pd.read_csv('StateDF.csv', header = 'infer') # Secondary dataframe of all states created by combining county data from the main dataframe
StateDF = StateDF.set_index('Index')
StateDF = StateDF.astype({'Date': np.datetime64})
US_DF = pd.read_csv('US_DF.csv', header = 'infer')     # US dataframe of all data created by combining state data from the state dataframe
US_DF = US_DF.set_index('Index')
US_DF = US_DF.astype({'Date': np.datetime64})

# Websites where csv files were downloaded from
# https://www.census.gov/data/datasets/time-series/demo/popest/2010s-counties-total.html#par_textimage_739801612
# https://www.kaggle.com/fireballbyedimyrnmom/us-counties-covid-19-dataset/data


In [4]:
# Set two dictionaries to convert between states and abreviations

get_st_full = {'AK': 'Alaska','AL': 'Alabama','AR': 'Arkansas','AZ': 'Arizona','CA': 'California','CO': 'Colorado','CT': 'Connecticut',
    'DC': 'District of Columbia','DE': 'Delaware','FL': 'Florida','GA': 'Georgia','HI': 'Hawaii','IA': 'Iowa','ID': 'Idaho','IL': 'Illinois',
    'IN': 'Indiana','KS': 'Kansas','KY': 'Kentucky','LA': 'Louisiana','MA': 'Massachusetts','MD': 'Maryland','ME': 'Maine','MI': 'Michigan',
    'MN': 'Minnesota','MO': 'Missouri','MS': 'Mississippi','MT': 'Montana','NC': 'North Carolina','ND': 'North Dakota','NE': 'Nebraska',
    'NH': 'New Hampshire','NJ': 'New Jersey','NM': 'New Mexico','NV': 'Nevada','NY': 'New York','OH': 'Ohio','OK': 'Oklahoma','OR': 'Oregon',
    'PA': 'Pennsylvania','RI': 'Rhode Island','SC': 'South Carolina','SD': 'South Dakota','TN': 'Tennessee','TX': 'Texas','UT': 'Utah',
    'VA': 'Virginia','VT': 'Vermont','WA': 'Washington','WI': 'Wisconsin','WV': 'West Virginia','WY': 'Wyoming'}

get_st_abr = {value:key for key, value in get_st_full.items()}

# Create list using MapChart website format for reference when creating dataframes
mapchartlist = ["North_Slope__AK","Northwest_Arctic__AK","Petersburg__AK","Prince_of_Wales_Hyder__AK","Ketchikan_Gateway__AK","Wrangell__AK","Sitka__AK","Haines__AK","Juneau__AK","Skagway__AK","Hoonah_Angoon__AK","Yakutat__AK","Aleutians_West__AK","Aleutians_East__AK","Kodiak_Island__AK","Bristol_Bay__AK","Dillingham__AK","Lake_and_Peninsula__AK","Kenai_Peninsula__AK","Anchorage__AK","Valdez-Cordova__AK","Matanuska-Susitna__AK","SE_Fairbanks__AK","Fairbanks_North_Star__AK","Denali__AK","Bethel__AK","Kusilvak__AK","Nome__AK","Yukon-Koyukuk__AK","Houston__AL","Henry__AL","Geneva__AL","Dale__AL","Coffee__AL","Covington__AL","Mobile__AL","Baldwin__AL","Escambia__AL","Conecuh__AL","Washington__AL","Choctaw__AL","Clarke__AL","Monroe__AL","Wilcox__AL","Butler__AL","Crenshaw__AL","Pike__AL","Bullock__AL","Macon__AL","Lowndes__AL","Montgomery__AL","Elmore__AL","Autauga__AL","Dallas__AL","Marengo__AL","Sumter__AL","Perry__AL","Chilton__AL","Coosa__AL","Tallapoosa__AL","Clay__AL","Talladega__AL","Shelby__AL","Bibb__AL","Hale__AL","Greene__AL","Pickens__AL","Tuscaloosa__AL","Jefferson__AL","Walker__AL","Lamar__AL","Fayette__AL","Franklin__AL","Marion__AL","Winston__AL","Cullman__AL","Blount__AL","St__Clair__AL","Calhoun__AL","Etowah__AL","Marshall__AL","Morgan__AL","Lawrence__AL","Colbert__AL","Lauderdale__AL","Limestone__AL","Madison__AL","Jackson__AL","DeKalb__AL","Cherokee__AL","Cleburne__AL","Randolph__AL","Chambers__AL","Lee__AL","Russell__AL","Barbour__AL","Chicot__AR","Ashley__AR","Pope__AR","Newton__AR","Madison__AR","Franklin__AR","Johnson__AR","Logan__AR","Yell__AR","Perry__AR","Conway__AR","Faulkner__AR","Van_Buren__AR","Searcy__AR","Stone__AR","Cleburne__AR","White__AR","Independence__AR","Jackson__AR","Woodruff__AR","St__Francis__AR","Crittenden__AR","Cross__AR","Poinsett__AR","Mississippi__AR","Craighead__AR","Izard__AR","Lawrence__AR","Sharp__AR","Greene__AR","Clay__AR","Randolph__AR","Fulton__AR","Baxter__AR","Marion__AR","Boone__AR","Carroll__AR","Benton__AR","Washington__AR","Crawford__AR","Sebastian__AR","Scott__AR","Montgomery__AR","Garland__AR","Saline__AR","Pulaski__AR","Lonoke__AR","Prairie__AR","Monroe__AR","Lee__AR","Phillips__AR","Jefferson__AR","Grant__AR","Hot_Spring__AR","Dallas__AR","Cleveland__AR","Lincoln__AR","Arkansas__AR","Desha__AR","Drew__AR","Bradley__AR","Calhoun__AR","Ouachita__AR","Nevada__AR","Hempstead__AR","Clark__AR","Pike__AR","Howard__AR","Polk__AR","Sevier__AR","Little_River__AR","Miller__AR","Lafayette__AR","Columbia__AR","Union__AR","Mohave__AZ","La_Paz__AZ","Cochise__AZ","Santa_Cruz__AZ","Pima__AZ","Greenlee__AZ","Graham__AZ","Apache__AZ","Navajo__AZ","Gila__AZ","Pinal__AZ","Yuma__AZ","Maricopa__AZ","Coconino__AZ","Yavapai__AZ","Del_Norte__CA","Siskiyou__CA","Imperial__CA","Riverside__CA","San_Diego__CA","Orange__CA","Los_Angeles__CA","Ventura__CA","Santa_Barbara__CA","San_Luis_Obispo__CA","Kern__CA","San_Bernardino__CA","Inyo__CA","Tulare__CA","Kings__CA","Madera__CA","Fresno__CA","Monterey__CA","San_Benito__CA","Merced__CA","Santa_Cruz__CA","San_Mateo__CA","Alameda__CA","Santa_Clara__CA","Stanislaus__CA","Mariposa__CA","Tuolumne__CA","Mono__CA","Alpine__CA","Calaveras__CA","Amador__CA","San_Joaquin__CA","Contra_Costa__CA","El_Dorado__CA","Sacramento__CA","Solano__CA","San_Francisco__CA","Marin__CA","Napa__CA","Sonoma__CA","Yolo__CA","Placer__CA","Nevada__CA","Sierra__CA","Yuba__CA","Sutter__CA","Colusa__CA","Lake__CA","Butte__CA","Glenn__CA","Plumas__CA","Tehama__CA","Mendocino__CA","Humboldt__CA","Trinity__CA","Shasta__CA","Lassen__CA","Modoc__CA","Montezuma__CO","La_Plata__CO","Prowers__CO","Bent__CO","Kiowa__CO","Cheyenne__CO","Kit_Carson__CO","Yuma__CO","Washington__CO","Phillips__CO","Sedgwick__CO","Logan__CO","Morgan__CO","Baca__CO","Otero__CO","Crowley__CO","Lincoln__CO","Elbert__CO","Arapahoe__CO","Weld__CO","Adams__CO","Denver__CO","Douglas__CO","El_Paso__CO","Teller__CO","Pueblo__CO","Las_Animas__CO","Huerfano__CO","Costilla__CO","Conejos__CO","Alamosa__CO","Rio_Grande__CO","Custer__CO","Fremont__CO","Saguache__CO","Chaffee__CO","Gunnison__CO","Pitkin__CO","Lake__CO","Park__CO","Jefferson__CO","Gilpin__CO","Clear_Creek__CO","Summit__CO","Eagle__CO","Boulder__CO","Grand__CO","Larimer__CO","Jackson__CO","Routt__CO","Moffat__CO","Rio_Blanco__CO","Garfield__CO","Delta__CO","Mesa__CO","Montrose__CO","San_Juan__CO","Dolores__CO","San_Miguel__CO","Ouray__CO","Hinsdale__CO","Mineral__CO","Archuleta__CO","Broomfield__CO","Hartford__CT","Litchfield__CT","Fairfield__CT","New_Haven__CT","Middlesex__CT","New_London__CT","Tolland__CT","Windham__CT","New_Castle__DE","Sussex__DE","Kent__DE","Bay__FL","Jackson__FL","Charlotte__FL","Lee__FL","Hendry__FL","Collier__FL","Monroe__FL","Miami-Dade__FL","Broward__FL","Palm_Beach__FL","Sarasota__FL","DeSoto__FL","Glades__FL","Martin__FL","St__Lucie__FL","Indian_River__FL","Okeechobee__FL","Highlands__FL","Hardee__FL","Manatee__FL","Pinellas__FL","Hillsborough__FL","Hernando__FL","Pasco__FL","Osceola__FL","Polk__FL","Orange__FL","Seminole__FL","Lake__FL","Sumter__FL","Citrus__FL","Levy__FL","Marion__FL","Volusia__FL","Brevard__FL","Washington__FL","Holmes__FL","Walton__FL","Okaloosa__FL","Santa_Rosa__FL","Escambia__FL","Calhoun__FL","Gulf__FL","Franklin__FL","Dixie__FL","Gilchrist__FL","Alachua__FL","Putnam__FL","Flagler__FL","St__Johns__FL","Clay__FL","Bradford__FL","Union__FL","Suwannee__FL","Lafayette__FL","Taylor__FL","Wakulla__FL","Liberty__FL","Gadsden__FL","Leon__FL","Jefferson__FL","Madison__FL","Hamilton__FL","Columbia__FL","Baker__FL","Duval__FL","Nassau__FL","Rabun__GA","Towns__GA","Mitchell__GA","Baker__GA","Miller__GA","Early__GA","Seminole__GA","Decatur__GA","Grady__GA","Thomas__GA","Brooks__GA","Colquitt__GA","Cook__GA","Lanier__GA","Lowndes__GA","Echols__GA","Clinch__GA","Atkinson__GA","Berrien__GA","Irwin__GA","Tift__GA","Worth__GA","Dougherty__GA","Calhoun__GA","Clay__GA","Quitman__GA","Randolph__GA","Terrell__GA","Lee__GA","Crisp__GA","Turner__GA","Ben_Hill__GA","Coffee__GA","Bacon__GA","Wayne__GA","Pierce__GA","Brantley__GA","Ware__GA","Charlton__GA","Camden__GA","Glynn__GA","McIntosh__GA","Heard__GA","Haralson__GA","Polk__GA","Floyd__GA","Chattooga__GA","Whitfield__GA","Catoosa__GA","Walker__GA","Dade__GA","Long__GA","Appling__GA","Jeff_Davis__GA","Stewart__GA","Webster__GA","Sumter__GA","Wilcox__GA","Telfair__GA","Wheeler__GA","Dodge__GA","Pulaski__GA","Dooly__GA","Macon__GA","Schley__GA","Marion__GA","Chattahoochee__GA","Muscogee__GA","Taylor__GA","Talbot__GA","Harris__GA","Troup__GA","Carroll__GA","Paulding__GA","Murray__GA","Gordon__GA","Bartow__GA","Cobb__GA","Douglas__GA","Coweta__GA","Meriwether__GA","Pike__GA","Upson__GA","Crawford__GA","Peach__GA","Houston__GA","Bleckley__GA","Laurens__GA","Wilkinson__GA","Twiggs__GA","Bibb__GA","Baldwin__GA","Jones__GA","Monroe__GA","Lamar__GA","Spalding__GA","Fayette__GA","Clayton__GA","Butts__GA","Henry__GA","DeKalb__GA","Fulton__GA","Cherokee__GA","Pickens__GA","Dawson__GA","Forsyth__GA","Gwinnett__GA","Rockdale__GA","Jasper__GA","Newton__GA","Walton__GA","Barrow__GA","Jackson__GA","Clarke__GA","Oconee__GA","Morgan__GA","Putnam__GA","Hancock__GA","Taliaferro__GA","Greene__GA","Oglethorpe__GA","Wilkes__GA","Warren__GA","McDuffie__GA","Glascock__GA","Jefferson__GA","Washington__GA","Johnson__GA","Treutlen__GA","Montgomery__GA","Toombs__GA","Liberty__GA","Evans__GA","Tattnall__GA","Candler__GA","Emanuel__GA","Chatham__GA","Bryan__GA","Bulloch__GA","Jenkins__GA","Effingham__GA","Screven__GA","Burke__GA","Richmond__GA","Columbia__GA","Lincoln__GA","Elbert__GA","Madison__GA","Banks__GA","Hall__GA","Lumpkin__GA","Gilmer__GA","Fannin__GA","Union__GA","White__GA","Habersham__GA","Hart__GA","Franklin__GA","Stephens__GA","Kauai__HI","Honolulu__HI","Maui__HI","Hawaii__HI","Kalawao__HI","Scott__IA","Clinton__IA","Lyon__IA","Sioux__IA","Plymouth__IA","Cherokee__IA","O_Brien__IA","Osceola__IA","Dickinson__IA","Clay__IA","Buena_Vista__IA","Pocahontas__IA","Palo_Alto__IA","Emmet__IA","Kossuth__IA","Humboldt__IA","Woodbury__IA","Ida__IA","Sac__IA","Calhoun__IA","Webster__IA","Greene__IA","Carroll__IA","Crawford__IA","Monona__IA","Harrison__IA","Shelby__IA","Audubon__IA","Guthrie__IA","Adair__IA","Cass__IA","Pottawattamie__IA","Fremont__IA","Mills__IA","Montgomery__IA","Page__IA","Taylor__IA","Adams__IA","Union__IA","Ringgold__IA","Decatur__IA","Clarke__IA","Madison__IA","Dallas__IA","Boone__IA","Hamilton__IA","Wright__IA","Hancock__IA","Winnebago__IA","Worth__IA","Cerro_Gordo__IA","Franklin__IA","Hardin__IA","Story__IA","Polk__IA","Warren__IA","Lucas__IA","Wayne__IA","Appanoose__IA","Monroe__IA","Marion__IA","Jasper__IA","Marshall__IA","Grundy__IA","Butler__IA","Floyd__IA","Mitchell__IA","Howard__IA","Chickasaw__IA","Bremer__IA","Black_Hawk__IA","Tama__IA","Poweshiek__IA","Mahaska__IA","Wapello__IA","Davis__IA","Van_Buren__IA","Jefferson__IA","Benton__IA","Iowa__IA","Keokuk__IA","Washington__IA","Henry__IA","Lee__IA","Des_Moines__IA","Louisa__IA","Muscatine__IA","Johnson__IA","Linn__IA","Cedar__IA","Jones__IA","Buchanan__IA","Fayette__IA","Winneshiek__IA","Allamakee__IA","Clayton__IA","Delaware__IA","Dubuque__IA","Jackson__IA","Fremont__ID","Clark__ID","Benewah__ID","Kootenai__ID","Bonner__ID","Boundary__ID","Latah__ID","Nez_Perce__ID","Lewis__ID","Adams__ID","Washington__ID","Payette__ID","Boise__ID","Gem__ID","Canyon__ID","Ada__ID","Elmore__ID","Owyhee__ID","Camas__ID","Gooding__ID","Jerome__ID","Twin_Falls__ID","Cassia__ID","Oneida__ID","Power__ID","Bannock__ID","Franklin__ID","Bear_Lake__ID","Caribou__ID","Minidoka__ID","Lincoln__ID","Bonneville__ID","Bingham__ID","Blaine__ID","Valley__ID","Shoshone__ID","Clearwater__ID","Idaho__ID","Custer__ID","Lemhi__ID","Butte__ID","Jefferson__ID","Madison__ID","Teton__ID","Vermilion__IL","Iroquois__IL","Alexander__IL","Pulaski__IL","Massac__IL","Hardin__IL","Pope__IL","Johnson__IL","Union__IL","Gallatin__IL","Saline__IL","Williamson__IL","Jackson__IL","Monroe__IL","Randolph__IL","Perry__IL","Franklin__IL","Hamilton__IL","White__IL","Jefferson__IL","Washington__IL","Clinton__IL","St__Clair__IL","Madison__IL","Bond__IL","Marion__IL","Wayne__IL","Wabash__IL","Edwards__IL","Lawrence__IL","Richland__IL","Clay__IL","Jasper__IL","Crawford__IL","Clark__IL","Cumberland__IL","Effingham__IL","Fayette__IL","Shelby__IL","Montgomery__IL","Jersey__IL","Macoupin__IL","Calhoun__IL","Greene__IL","Christian__IL","Coles__IL","Edgar__IL","Douglas__IL","Moultrie__IL","Macon__IL","Sangamon__IL","Morgan__IL","Scott__IL","Pike__IL","Brown__IL","Adams__IL","Hancock__IL","McDonough__IL","Schuyler__IL","Cass__IL","Menard__IL","Mason__IL","Logan__IL","De_Witt__IL","Piatt__IL","Champaign__IL","Ford__IL","McLean__IL","Tazewell__IL","Woodford__IL","Peoria__IL","Fulton__IL","Warren__IL","Henderson__IL","Mercer__IL","Knox__IL","Stark__IL","Marshall__IL","Putnam__IL","Livingston__IL","Kankakee__IL","Grundy__IL","Will__IL","Kendall__IL","La_Salle__IL","Bureau__IL","Henry__IL","Rock_Island__IL","Lee__IL","Whiteside__IL","Carroll__IL","Ogle__IL","DeKalb__IL","Kane__IL","DuPage__IL","Cook__IL","Lake__IL","McHenry__IL","Boone__IL","Winnebago__IL","Stephenson__IL","Jo_Daviess__IL","Porter__IN","Lake__IN","Perry__IN","Spencer__IN","Warrick__IN","Vanderburgh__IN","Posey__IN","Gibson__IN","Pike__IN","Dubois__IN","Crawford__IN","Harrison__IN","Floyd__IN","Clark__IN","Ohio__IN","Switzerland__IN","Jefferson__IN","Scott__IN","Washington__IN","Orange__IN","Martin__IN","Daviess__IN","Knox__IN","Sullivan__IN","Greene__IN","Lawrence__IN","Jackson__IN","Jennings__IN","Ripley__IN","Dearborn__IN","Franklin__IN","Decatur__IN","Bartholomew__IN","Brown__IN","Monroe__IN","Owen__IN","Clay__IN","Vigo__IN","Vermillion__IN","Parke__IN","Putnam__IN","Morgan__IN","Johnson__IN","Shelby__IN","Rush__IN","Fayette__IN","Union__IN","Wayne__IN","Henry__IN","Hancock__IN","Marion__IN","Hendricks__IN","Boone__IN","Hamilton__IN","Randolph__IN","Delaware__IN","Madison__IN","Tipton__IN","Clinton__IN","Montgomery__IN","Fountain__IN","Jay__IN","Blackford__IN","Grant__IN","Howard__IN","Carroll__IN","Tippecanoe__IN","Warren__IN","Benton__IN","Adams__IN","Wells__IN","Huntington__IN","Wabash__IN","Miami__IN","Cass__IN","White__IN","Allen__IN","Whitley__IN","Fulton__IN","Pulaski__IN","DeKalb__IN","Noble__IN","Kosciusko__IN","Marshall__IN","Starke__IN","Jasper__IN","Newton__IN","Steuben__IN","LaGrange__IN","Elkhart__IN","St__Joseph__IN","LaPorte__IN","Doniphan__KS","Brown__KS","Stanton__KS","Morton__KS","Stevens__KS","Grant__KS","Haskell__KS","Seward__KS","Meade__KS","Gray__KS","Ford__KS","Clark__KS","Comanche__KS","Kiowa__KS","Edwards__KS","Pratt__KS","Barber__KS","Harper__KS","Kingman__KS","Reno__KS","Harvey__KS","Sedgwick__KS","Sumner__KS","Cowley__KS","Butler__KS","Chautauqua__KS","Elk__KS","Greenwood__KS","Montgomery__KS","Wilson__KS","Woodson__KS","Labette__KS","Neosho__KS","Allen__KS","Cherokee__KS","Crawford__KS","Bourbon__KS","Hamilton__KS","Greeley__KS","Wallace__KS","Sherman__KS","Cheyenne__KS","Stafford__KS","Kearny__KS","Finney__KS","Hodgeman__KS","Pawnee__KS","Coffey__KS","Anderson__KS","Linn__KS","Miami__KS","Franklin__KS","Osage__KS","Lyon__KS","Chase__KS","Morris__KS","Marion__KS","McPherson__KS","Rice__KS","Saline__KS","Ellsworth__KS","Barton__KS","Rush__KS","Ness__KS","Lane__KS","Scott__KS","Wichita__KS","Logan__KS","Gove__KS","Trego__KS","Ellis__KS","Russell__KS","Lincoln__KS","Ottawa__KS","Dickinson__KS","Geary__KS","Wabaunsee__KS","Shawnee__KS","Douglas__KS","Johnson__KS","Wyandotte__KS","Leavenworth__KS","Jefferson__KS","Atchison__KS","Jackson__KS","Pottawatomie__KS","Riley__KS","Clay__KS","Cloud__KS","Mitchell__KS","Osborne__KS","Rooks__KS","Graham__KS","Sheridan__KS","Thomas__KS","Rawlins__KS","Decatur__KS","Phillips__KS","Norton__KS","Smith__KS","Jewell__KS","Republic__KS","Washington__KS","Marshall__KS","Nemaha__KS","Hickman__KY","Fulton__KY","Bell__KY","Knox__KY","Whitley__KY","Laurel__KY","Clay__KY","Leslie__KY","Harlan__KY","Letcher__KY","Knott__KY","Perry__KY","Owsley__KY","Jackson__KY","Estill__KY","Lee__KY","Breathitt__KY","Floyd__KY","Johnson__KY","Magoffin__KY","Wolfe__KY","Powell__KY","Clark__KY","Montgomery__KY","Menifee__KY","Morgan__KY","Elliott__KY","Rowan__KY","Bath__KY","Nicholas__KY","Fleming__KY","Robertson__KY","Bracken__KY","Mason__KY","Lewis__KY","Boyd__KY","Greenup__KY","Carter__KY","Lawrence__KY","Martin__KY","Pike__KY","Harrison__KY","Bourbon__KY","Madison__KY","Rockcastle__KY","McCreary__KY","Lincoln__KY","Garrard__KY","Jessamine__KY","Fayette__KY","Woodford__KY","Scott__KY","Franklin__KY","Anderson__KY","Mercer__KY","Boyle__KY","Casey__KY","Pulaski__KY","Wayne__KY","Russell__KY","Clinton__KY","Cumberland__KY","Adair__KY","Taylor__KY","Marion__KY","Washington__KY","Spencer__KY","Shelby__KY","Henry__KY","Owen__KY","Grant__KY","Pendleton__KY","Campbell__KY","Kenton__KY","Boone__KY","Gallatin__KY","Carroll__KY","Trimble__KY","Oldham__KY","Jefferson__KY","Bullitt__KY","Nelson__KY","Larue__KY","Green__KY","Metcalfe__KY","Monroe__KY","Simpson__KY","Allen__KY","Barren__KY","Hart__KY","Hardin__KY","Meade__KY","Hancock__KY","Breckinridge__KY","Grayson__KY","Edmonson__KY","Warren__KY","Butler__KY","Logan__KY","Todd__KY","Muhlenberg__KY","Ohio__KY","Daviess__KY","McLean__KY","Hopkins__KY","Christian__KY","Trigg__KY","Lyon__KY","Caldwell__KY","Webster__KY","Henderson__KY","Union__KY","Crittenden__KY","Livingston__KY","Calloway__KY","Marshall__KY","Graves__KY","McCracken__KY","Carlisle__KY","Ballard__KY","St__Tammany__LA","Washington__LA","Red_River__LA","Bienville__LA","Lincoln__LA","Jackson__LA","Winn__LA","Caldwell__LA","Ouachita__LA","Richland__LA","Franklin__LA","Tensas__LA","Madison__LA","East_Carroll__LA","West_Carroll__LA","Morehouse__LA","Union__LA","Claiborne__LA","Webster__LA","Bossier__LA","Caddo__LA","De_Soto__LA","Sabine__LA","Natchitoches__LA","Grant__LA","LaSalle__LA","Catahoula__LA","Concordia__LA","Avoyelles__LA","Rapides__LA","Vernon__LA","Evangeline__LA","Allen__LA","Beauregard__LA","Jefferson_Davis__LA","Calcasieu__LA","Cameron__LA","Acadia__LA","St__Landry__LA","Pointe_Coupee__LA","West_Baton_Rouge__LA","Lafayette__LA","Vermilion__LA","Iberia__LA","Iberville__LA","Assumption__LA","St__Martin__LA","St__Mary__LA","Terrebonne__LA","Lafourche__LA","St__James__LA","Ascension__LA","East_Baton_Rouge__LA","East_Feliciana__LA","St__Helena__LA","Livingston__LA","St__John_the_Baptist__LA","St__Charles__LA","Tangipahoa__LA","Orleans__LA","St__Bernard__LA","Plaquemines__LA","West_Feliciana__LA","Jefferson__LA","Worcester__MA","Middlesex__MA","Nantucket__MA","Barnstable__MA","Suffolk__MA","Bristol__MA","Plymouth__MA","Norfolk__MA","Essex__MA","Hampden__MA","Hampshire__MA","Berkshire__MA","Franklin__MA","Dukes__MA","Allegany__MD","Garrett__MD","St_Mary_s__MD","Charles__MD","Worcester__MD","Wicomico__MD","Washington__MD","Calvert__MD","Talbot__MD","Caroline__MD","Kent__MD","Baltimore_City__MD","Prince_George_s__MD","Anne_Arundel__MD","Howard__MD","Montgomery__MD","Queen_Anne_s__MD","Cecil__MD","Harford__MD","Baltimore_County__MD","Carroll__MD","Frederick__MD","Somerset__MD","Dorchester__MD","Aroostook__ME","Piscataquis__ME","Knox__ME","Lincoln__ME","Sagadahoc__ME","Cumberland__ME","York__ME","Androscoggin__ME","Oxford__ME","Kennebec__ME","Waldo__ME","Hancock__ME","Washington__ME","Penobscot__ME","Franklin__ME","Somerset__ME","Gogebic__MI","Ontonagon__MI","Monroe__MI","Lenawee__MI","Hillsdale__MI","Branch__MI","St__Joseph__MI","Cass__MI","Berrien__MI","Wayne__MI","Washtenaw__MI","Jackson__MI","Calhoun__MI","Kalamazoo__MI","Van_Buren__MI","Allegan__MI","Barry__MI","Eaton__MI","Ingham__MI","Livingston__MI","Oakland__MI","Macomb__MI","St__Clair__MI","Lapeer__MI","Genesee__MI","Shiawassee__MI","Clinton__MI","Ionia__MI","Ottawa__MI","Muskegon__MI","Kent__MI","Montcalm__MI","Gratiot__MI","Saginaw__MI","Sanilac__MI","Tuscola__MI","Huron__MI","Bay__MI","Midland__MI","Isabella__MI","Mecosta__MI","Newaygo__MI","Oceana__MI","Arenac__MI","Gladwin__MI","Clare__MI","Osceola__MI","Lake__MI","Mason__MI","Iosco__MI","Ogemaw__MI","Roscommon__MI","Missaukee__MI","Wexford__MI","Manistee__MI","Alcona__MI","Oscoda__MI","Crawford__MI","Kalkaska__MI","Grand_Traverse__MI","Benzie__MI","Leelanau__MI","Alpena__MI","Montmorency__MI","Otsego__MI","Antrim__MI","Charlevoix__MI","Presque_Isle__MI","Cheboygan__MI","Emmet__MI","Mackinac__MI","Chippewa__MI","Luce__MI","Schoolcraft__MI","Alger__MI","Delta__MI","Menominee__MI","Dickinson__MI","Marquette__MI","Iron__MI","Keweenaw__MI","Houghton__MI","Baraga__MI","Houston__MN","Fillmore__MN","Koochiching__MN","Lake_of_the_Woods__MN","Roseau__MN","Kittson__MN","Red_Lake__MN","Pennington__MN","Marshall__MN","Polk__MN","Norman__MN","Mahnomen__MN","Clearwater__MN","Beltrami__MN","Cook__MN","Lake__MN","Itasca__MN","Hubbard__MN","Cass__MN","Wadena__MN","Becker__MN","Clay__MN","Wilkin__MN","Otter_Tail__MN","St__Louis__MN","Carlton__MN","Pine__MN","Kanabec__MN","Aitkin__MN","Crow_Wing__MN","Morrison__MN","Todd__MN","Douglas__MN","Grant__MN","Traverse__MN","Big_Stone__MN","Swift__MN","Stevens__MN","Pope__MN","Stearns__MN","Benton__MN","Mille_Lacs__MN","Isanti__MN","Chisago__MN","Ramsey__MN","Anoka__MN","Sherburne__MN","Wright__MN","Meeker__MN","Kandiyohi__MN","Chippewa__MN","Washington__MN","Dakota__MN","Hennepin__MN","Scott__MN","Carver__MN","McLeod__MN","Sibley__MN","Renville__MN","Lac_qui_Parle__MN","Yellow_Medicine__MN","Lincoln__MN","Lyon__MN","Redwood__MN","Brown__MN","Nicollet__MN","Le_Sueur__MN","Rice__MN","Goodhue__MN","Wabasha__MN","Winona__MN","Olmsted__MN","Dodge__MN","Steele__MN","Waseca__MN","Blue_Earth__MN","Watonwan__MN","Cottonwood__MN","Murray__MN","Pipestone__MN","Rock__MN","Nobles__MN","Jackson__MN","Martin__MN","Faribault__MN","Freeborn__MN","Mower__MN","New_Madrid__MO","Pemiscot__MO","Atchison__MO","Holt__MO","Platte__MO","Buchanan__MO","Andrew__MO","Nodaway__MO","Worth__MO","Gentry__MO","DeKalb__MO","Clinton__MO","Clay__MO","Caldwell__MO","Daviess__MO","Harrison__MO","Mercer__MO","Grundy__MO","Livingston__MO","Carroll__MO","Ray__MO","Lafayette__MO","Jackson__MO","Johnson__MO","Henry__MO","Cass__MO","St__Clair__MO","Bates__MO","Cedar__MO","Vernon__MO","Dade__MO","Barton__MO","Jasper__MO","Newton__MO","McDonald__MO","Lawrence__MO","Barry__MO","Polk__MO","Stone__MO","Greene__MO","Taney__MO","Christian__MO","Webster__MO","Dallas__MO","Hickory__MO","Benton__MO","Pettis__MO","Saline__MO","Chariton__MO","Linn__MO","Sullivan__MO","Putnam__MO","Schuyler__MO","Adair__MO","Macon__MO","Randolph__MO","Howard__MO","Cooper__MO","Boone__MO","Cole__MO","Moniteau__MO","Morgan__MO","Miller__MO","Camden__MO","Pulaski__MO","Laclede__MO","Wright__MO","Douglas__MO","Ozark__MO","Howell__MO","Texas__MO","Phelps__MO","Maries__MO","Osage__MO","Callaway__MO","Monroe__MO","Shelby__MO","Knox__MO","Scotland__MO","Clark__MO","Lewis__MO","Marion__MO","Ralls__MO","Audrain__MO","Montgomery__MO","Pike__MO","Lincoln__MO","Warren__MO","Gasconade__MO","Crawford__MO","Dent__MO","Shannon__MO","Oregon__MO","Ripley__MO","Carter__MO","Reynolds__MO","Iron__MO","Washington__MO","Franklin__MO","St__Louis__MO","St__Charles__MO","St__Louis_Co___MO","Jefferson__MO","St__Francois__MO","Sainte_Genevieve__MO","Madison__MO","Wayne__MO","Butler__MO","Dunklin__MO","Stoddard__MO","Bollinger__MO","Perry__MO","Cape_Girardeau__MO","Scott__MO","Mississippi__MO","Harrison__MS","Jackson__MS","Grenada__MS","Yalobusha__MS","Clay__MS","Webster__MS","Chickasaw__MS","Calhoun__MS","Pontotoc__MS","Lee__MS","Union__MS","Lafayette__MS","Tate__MS","Panola__MS","Quitman__MS","Tallahatchie__MS","Leflore__MS","Sunflower__MS","Humphreys__MS","Holmes__MS","Carroll__MS","Montgomery__MS","Choctaw__MS","Oktibbeha__MS","Lowndes__MS","Monroe__MS","Itawamba__MS","Prentiss__MS","Tishomingo__MS","Alcorn__MS","Tippah__MS","Benton__MS","Marshall__MS","DeSoto__MS","Tunica__MS","Coahoma__MS","Bolivar__MS","Washington__MS","Sharkey__MS","Issaquena__MS","Yazoo__MS","Attala__MS","Winston__MS","Noxubee__MS","Kemper__MS","Neshoba__MS","Leake__MS","Madison__MS","Rankin__MS","Hinds__MS","Warren__MS","Jefferson__MS","Claiborne__MS","Copiah__MS","Simpson__MS","Smith__MS","Scott__MS","Newton__MS","Lauderdale__MS","Clarke__MS","Jasper__MS","Jones__MS","Covington__MS","Jefferson_Davis__MS","Lawrence__MS","Lincoln__MS","Franklin__MS","Adams__MS","Wilkinson__MS","Amite__MS","Pike__MS","Walthall__MS","Marion__MS","Lamar__MS","Forrest__MS","Wayne__MS","Greene__MS","Perry__MS","Stone__MS","Pearl_River__MS","Hancock__MS","George__MS","Sheridan__MT","Roosevelt__MT","Gallatin__MT","Madison__MT","Broadwater__MT","Jefferson__MT","Silver_Bow__MT","Deer_Lodge__MT","Beaverhead__MT","Ravalli__MT","Granite__MT","Lewis_and_Clark__MT","Powell__MT","Lake__MT","Missoula__MT","Mineral__MT","Sanders__MT","Lincoln__MT","Flathead__MT","Glacier__MT","Liberty__MT","Toole__MT","Pondera__MT","Teton__MT","Cascade__MT","Meagher__MT","Park__MT","Hill__MT","Chouteau__MT","Judith_Basin__MT","Wheatland__MT","Sweet_Grass__MT","Carbon__MT","Stillwater__MT","Golden_Valley__MT","Fergus__MT","Blaine__MT","Phillips__MT","Petroleum__MT","Musselshell__MT","Yellowstone__MT","Big_Horn__MT","Treasure__MT","Rosebud__MT","Garfield__MT","Daniels__MT","Valley__MT","McCone__MT","Prairie__MT","Powder_River__MT","Custer__MT","Carter__MT","Fallon__MT","Wibaux__MT","Dawson__MT","Richland__MT","Clay__NC","Macon__NC","Transylvania__NC","Jackson__NC","Haywood__NC","Swain__NC","Graham__NC","Cherokee__NC","Mitchell__NC","Yancey__NC","Madison__NC","Buncombe__NC","Henderson__NC","Polk__NC","Rutherford__NC","McDowell__NC","Avery__NC","Watauga__NC","Alexander__NC","Ashe__NC","Alleghany__NC","Wilkes__NC","Caldwell__NC","Burke__NC","Cleveland__NC","Gaston__NC","Lincoln__NC","Catawba__NC","Iredell__NC","Mecklenburg__NC","Rowan__NC","Davie__NC","Yadkin__NC","Surry__NC","Stokes__NC","Forsyth__NC","Davidson__NC","Stanly__NC","Cabarrus__NC","Union__NC","Anson__NC","Rockingham__NC","Guilford__NC","Randolph__NC","Montgomery__NC","Richmond__NC","Scotland__NC","Caswell__NC","Alamance__NC","Moore__NC","Hoke__NC","Robeson__NC","Columbus__NC","Brunswick__NC","New_Hanover__NC","Bladen__NC","Cumberland__NC","Chatham__NC","Harnett__NC","Lee__NC","Orange__NC","Person__NC","Durham__NC","Pender__NC","Sampson__NC","Wayne__NC","Johnston__NC","Wake__NC","Granville__NC","Vance__NC","Warren__NC","Franklin__NC","Nash__NC","Northampton__NC","Halifax__NC","Edgecombe__NC","Wilson__NC","Greene__NC","Lenoir__NC","Jones__NC","Duplin__NC","Onslow__NC","Carteret__NC","Pamlico__NC","Craven__NC","Pitt__NC","Martin__NC","Chowan__NC","Hertford__NC","Bertie__NC","Gates__NC","Camden__NC","Pasquotank__NC","Perquimans__NC","Beaufort__NC","Washington__NC","Tyrrell__NC","Currituck__NC","Dare__NC","Hyde__NC","Cass__ND","Richland__ND","Burke__ND","Divide__ND","Williams__ND","Mountrail__ND","McKenzie__ND","Dunn__ND","Stark__ND","Golden_Valley__ND","Billings__ND","Bowman__ND","Slope__ND","Hettinger__ND","Adams__ND","Mercer__ND","Grant__ND","Sioux__ND","Morton__ND","Oliver__ND","Renville__ND","Ward__ND","McLean__ND","Bottineau__ND","McHenry__ND","Sheridan__ND","Burleigh__ND","Emmons__ND","McIntosh__ND","Logan__ND","Kidder__ND","Wells__ND","Pierce__ND","Rolette__ND","Towner__ND","Benson__ND","Eddy__ND","Foster__ND","Stutsman__ND","LaMoure__ND","Dickey__ND","Sargent__ND","Ransom__ND","Barnes__ND","Griggs__ND","Steele__ND","Nelson__ND","Ramsey__ND","Cavalier__ND","Pembina__ND","Walsh__ND","Grand_Forks__ND","Traill__ND","Dawes__NE","Sioux__NE","Richardson__NE","Pawnee__NE","Nemaha__NE","Johnson__NE","Otoe__NE","Cass__NE","Sarpy__NE","Douglas__NE","Washington__NE","Burt__NE","Thurston__NE","Dakota__NE","Dixon__NE","Wayne__NE","Pierce__NE","Cedar__NE","Knox__NE","Dodge__NE","Colfax__NE","Cuming__NE","Stanton__NE","Madison__NE","Antelope__NE","Boone__NE","Platte__NE","Saunders__NE","Seward__NE","Butler__NE","Polk__NE","York__NE","Hamilton__NE","Nance__NE","Merrick__NE","Hall__NE","Howard__NE","Sherman__NE","Boyd__NE","Holt__NE","Valley__NE","Greeley__NE","Wheeler__NE","Garfield__NE","Loup__NE","Rock__NE","Keya_Paha__NE","Brown__NE","Blaine__NE","Custer__NE","Buffalo__NE","Dawson__NE","Phelps__NE","Kearney__NE","Adams__NE","Clay__NE","Fillmore__NE","Saline__NE","Lancaster__NE","Gage__NE","Jefferson__NE","Thayer__NE","Nuckolls__NE","Webster__NE","Franklin__NE","Harlan__NE","Gosper__NE","Furnas__NE","Red_Willow__NE","Hitchcock__NE","Dundy__NE","Frontier__NE","Hayes__NE","Chase__NE","Perkins__NE","Lincoln__NE","Keith__NE","Logan__NE","McPherson__NE","Arthur__NE","Thomas__NE","Hooker__NE","Grant__NE","Deuel__NE","Garden__NE","Cheyenne__NE","Morrill__NE","Kimball__NE","Banner__NE","Scotts_Bluff__NE","Box_Butte__NE","Cherry__NE","Sheridan__NE","Coos__NH","Grafton__NH","Strafford__NH","Rockingham__NH","Cheshire__NH","Sullivan__NH","Hillsborough__NH","Belknap__NH","Merrimack__NH","Carroll__NH","Monmouth__NJ","Ocean__NJ","Essex__NJ","Union__NJ","Hudson__NJ","Bergen__NJ","Middlesex__NJ","Somerset__NJ","Morris__NJ","Passaic__NJ","Sussex__NJ","Warren__NJ","Hunterdon__NJ","Mercer__NJ","Camden__NJ","Gloucester__NJ","Salem__NJ","Cumberland__NJ","Cape_May__NJ","Atlantic__NJ","Burlington__NJ","Richmond__NY","Clinton__NY","Essex__NY","Orleans__NY","Genesee__NY","Wyoming__NY","Livingston__NY","Schenectady__NY","Montgomery__NY","Fulton__NY","Seneca__NY","Ontario__NY","Yates__NY","Schuyler__NY","Tompkins__NY","Cayuga__NY","Onondaga__NY","Cortland__NY","Chenango__NY","Madison__NY","Oneida__NY","Lewis__NY","Herkimer__NY","Otsego__NY","Schoharie__NY","Albany__NY","Saratoga__NY","Warren__NY","Hamilton__NY","Franklin__NY","St__Lawrence__NY","Jefferson__NY","Oswego__NY","Wayne__NY","Monroe__NY","Niagara__NY","Erie__NY","Chautauqua__NY","Cattaraugus__NY","Allegany__NY","Steuben__NY","Chemung__NY","Tioga__NY","Broome__NY","Delaware__NY","Sullivan__NY","Greene__NY","New_York__NY","Ulster__NY","Bronx__NY","Kings__NY","Rockland__NY","Nassau__NY","Orange__NY","Queens__NY","Suffolk__NY","Westchester__NY","Putnam__NY","Dutchess__NY","Columbia__NY","Rensselaer__NY","Washington__NY","Lea__NM","Eddy__NM","Los_Alamos__NM","Rio_Arriba__NM","San_Juan__NM","McKinley__NM","Bernalillo__NM","Valencia__NM","Cibola__NM","Sierra__NM","Socorro__NM","Catron__NM","Grant__NM","Hidalgo__NM","Luna__NM","Dona_Ana__NM","Taos__NM","Santa_Fe__NM","Otero__NM","Torrance__NM","Lincoln__NM","Colfax__NM","Mora__NM","Union__NM","Harding__NM","San_Miguel__NM","Quay__NM","Guadalupe__NM","De_Baca__NM","Curry__NM","Roosevelt__NM","Chaves__NM","Sandoval__NM","Elko__NV","White_Pine__NV","Clark__NV","Lincoln__NV","Esmeralda__NV","Nye__NV","Eureka__NV","Lander__NV","Mineral__NV","Douglas__NV","Carson_City__NV","Lyon__NV","Churchill__NV","Pershing__NV","Storey__NV","Washoe__NV","Humboldt__NV","Trumbull__OH","Ashtabula__OH","Lawrence__OH","Scioto__OH","Adams__OH","Pike__OH","Jackson__OH","Gallia__OH","Meigs__OH","Vinton__OH","Washington__OH","Athens__OH","Hocking__OH","Ross__OH","Highland__OH","Brown__OH","Clermont__OH","Hamilton__OH","Clinton__OH","Warren__OH","Butler__OH","Preble__OH","Montgomery__OH","Greene__OH","Fayette__OH","Pickaway__OH","Fairfield__OH","Perry__OH","Monroe__OH","Noble__OH","Morgan__OH","Muskingum__OH","Belmont__OH","Harrison__OH","Guernsey__OH","Coshocton__OH","Licking__OH","Delaware__OH","Franklin__OH","Madison__OH","Clark__OH","Champaign__OH","Miami__OH","Shelby__OH","Darke__OH","Mercer__OH","Auglaize__OH","Logan__OH","Union__OH","Marion__OH","Morrow__OH","Knox__OH","Holmes__OH","Tuscarawas__OH","Carroll__OH","Jefferson__OH","Columbiana__OH","Stark__OH","Wayne__OH","Ashland__OH","Richland__OH","Crawford__OH","Wyandot__OH","Hardin__OH","Allen__OH","Hancock__OH","Putnam__OH","Van_Wert__OH","Mahoning__OH","Portage__OH","Summit__OH","Medina__OH","Huron__OH","Seneca__OH","Sandusky__OH","Wood__OH","Henry__OH","Paulding__OH","Defiance__OH","Geauga__OH","Lake__OH","Cuyahoga__OH","Lorain__OH","Erie__OH","Ottawa__OH","Lucas__OH","Fulton__OH","Williams__OH","Texas__OK","Cimarron__OK","Ottawa__OK","Craig__OK","Mayes__OK","Delaware__OK","Adair__OK","Cherokee__OK","Sequoyah__OK","Haskell__OK","Le_Flore__OK","Latimer__OK","Pittsburg__OK","Coal__OK","Atoka__OK","Pushmataha__OK","McCurtain__OK","Choctaw__OK","Bryan__OK","Marshall__OK","Johnston__OK","Pontotoc__OK","Garvin__OK","Murray__OK","Love__OK","Carter__OK","Jefferson__OK","Stephens__OK","Cotton__OK","Comanche__OK","Tillman__OK","Jackson__OK","Greer__OK","Kiowa__OK","Harmon__OK","Muskogee__OK","McIntosh__OK","Hughes__OK","Seminole__OK","Pottawatomie__OK","Cleveland__OK","McClain__OK","Grady__OK","Caddo__OK","Canadian__OK","Oklahoma__OK","Lincoln__OK","Okfuskee__OK","Okmulgee__OK","Wagoner__OK","Rogers__OK","Nowata__OK","Washington__OK","Tulsa__OK","Creek__OK","Payne__OK","Logan__OK","Kingfisher__OK","Blaine__OK","Washita__OK","Beckham__OK","Custer__OK","Dewey__OK","Roger_Mills__OK","Ellis__OK","Woodward__OK","Major__OK","Garfield__OK","Noble__OK","Pawnee__OK","Osage__OK","Kay__OK","Grant__OK","Alfalfa__OK","Woods__OK","Harper__OK","Beaver__OK","Clatsop__OR","Columbia__OR","Curry__OR","Coos__OR","Josephine__OR","Jackson__OR","Douglas__OR","Klamath__OR","Lake__OR","Harney__OR","Malheur__OR","Baker__OR","Wallowa__OR","Union__OR","Grant__OR","Wheeler__OR","Crook__OR","Deschutes__OR","Jefferson__OR","Linn__OR","Lane__OR","Benton__OR","Lincoln__OR","Polk__OR","Yamhill__OR","Marion__OR","Clackamas__OR","Umatilla__OR","Morrow__OR","Gilliam__OR","Sherman__OR","Wasco__OR","Hood_River__OR","Multnomah__OR","Washington__OR","Tillamook__OR","Philadelphia__PA","Bucks__PA","Greene__PA","Fayette__PA","Washington__PA","Allegheny__PA","Beaver__PA","Lawrence__PA","Butler__PA","Armstrong__PA","Clarion__PA","Forest__PA","Venango__PA","Mercer__PA","Crawford__PA","Erie__PA","Westmoreland__PA","Indiana__PA","Jefferson__PA","Warren__PA","McKean__PA","Cameron__PA","Elk__PA","Clearfield__PA","Blair__PA","Cambria__PA","Somerset__PA","Bedford__PA","Fulton__PA","Huntingdon__PA","Franklin__PA","Mifflin__PA","Potter__PA","Clinton__PA","Centre__PA","Union__PA","Montour__PA","Tioga__PA","Lycoming__PA","Wyoming__PA","Sullivan__PA","Columbia__PA","Northumberland__PA","Snyder__PA","Juniata__PA","Perry__PA","Cumberland__PA","Adams__PA","York__PA","Dauphin__PA","Lebanon__PA","Lancaster__PA","Berks__PA","Schuylkill__PA","Carbon__PA","Luzerne__PA","Lackawanna__PA","Bradford__PA","Susquehanna__PA","Wayne__PA","Pike__PA","Monroe__PA","Northampton__PA","Lehigh__PA","Delaware__PA","Montgomery__PA","Chester__PA","Kent__RI","Providence__RI","Bristol__RI","Newport__RI","Washington__RI","Pickens__SC","Oconee__SC","Beaufort__SC","Hampton__SC","Jasper__SC","Marion__SC","Dillon__SC","Horry__SC","Charleston__SC","Dorchester__SC","Berkeley__SC","Georgetown__SC","Clarendon__SC","Williamsburg__SC","Florence__SC","Marlboro__SC","Darlington__SC","Lee__SC","Kershaw__SC","Sumter__SC","Calhoun__SC","Colleton__SC","Allendale__SC","Bamberg__SC","Orangeburg__SC","Barnwell__SC","Aiken__SC","Lexington__SC","Richland__SC","Fairfield__SC","Newberry__SC","Saluda__SC","Edgefield__SC","McCormick__SC","Greenwood__SC","Laurens__SC","Abbeville__SC","Anderson__SC","Union__SC","Chesterfield__SC","Lancaster__SC","Chester__SC","York__SC","Cherokee__SC","Spartanburg__SC","Greenville__SC","Perkins__SD","Harding__SD","Custer__SD","Fall_River__SD","Oglala_Lakota__SD","Bennett__SD","Mellette__SD","Todd__SD","Tripp__SD","Gregory__SD","Brule__SD","Aurora__SD","Davison__SD","Douglas__SD","Charles_Mix__SD","Bon_Homme__SD","Yankton__SD","Hutchinson__SD","Hanson__SD","McCook__SD","Turner__SD","Clay__SD","Union__SD","Lincoln__SD","Minnehaha__SD","Lyman__SD","Jones__SD","Jackson__SD","Stanley__SD","Haakon__SD","Pennington__SD","Lawrence__SD","Butte__SD","Meade__SD","Ziebach__SD","Dewey__SD","Walworth__SD","Potter__SD","Sully__SD","Hughes__SD","Buffalo__SD","Jerauld__SD","Sanborn__SD","Miner__SD","Lake__SD","Moody__SD","Brookings__SD","Hamlin__SD","Kingsbury__SD","Beadle__SD","Hyde__SD","Hand__SD","Edmunds__SD","Faulk__SD","Spink__SD","Clark__SD","Codington__SD","Deuel__SD","Grant__SD","Day__SD","Roberts__SD","Marshall__SD","Brown__SD","McPherson__SD","Campbell__SD","Corson__SD","Sullivan__TN","Johnson__TN","McNairy__TN","Chester__TN","Hardeman__TN","Fayette__TN","Haywood__TN","Madison__TN","Crockett__TN","Gibson__TN","Carroll__TN","Weakley__TN","Obion__TN","Lake__TN","Dyer__TN","Lauderdale__TN","Tipton__TN","Shelby__TN","Henderson__TN","Henry__TN","Benton__TN","Decatur__TN","Hardin__TN","Lawrence__TN","Wayne__TN","Lewis__TN","Perry__TN","Hickman__TN","Humphreys__TN","Houston__TN","Stewart__TN","Montgomery__TN","Robertson__TN","Cheatham__TN","Dickson__TN","Marshall__TN","Williamson__TN","Giles__TN","Maury__TN","Lincoln__TN","Moore__TN","Bedford__TN","Rutherford__TN","Davidson__TN","Sumner__TN","Wilson__TN","Cannon__TN","Coffee__TN","Franklin__TN","Marion__TN","Sequatchie__TN","Grundy__TN","Van_Buren__TN","Warren__TN","White__TN","DeKalb__TN","Putnam__TN","Trousdale__TN","Macon__TN","Smith__TN","Jackson__TN","Overton__TN","Clay__TN","Pickett__TN","Bledsoe__TN","Fentress__TN","Cumberland__TN","Rhea__TN","Meigs__TN","Hamilton__TN","Bradley__TN","Polk__TN","Monroe__TN","McMinn__TN","Roane__TN","Morgan__TN","Scott__TN","Campbell__TN","Anderson__TN","Loudon__TN","Blount__TN","Sevier__TN","Knox__TN","Union__TN","Claiborne__TN","Hancock__TN","Grainger__TN","Hamblen__TN","Jefferson__TN","Cocke__TN","Unicoi__TN","Hawkins__TN","Greene__TN","Washington__TN","Carter__TN","Bowie__TX","Red_River__TX","Crosby__TX","Lubbock__TX","Hockley__TX","Cochran__TX","Floyd__TX","Hale__TX","Lamb__TX","Bailey__TX","Briscoe__TX","Swisher__TX","Castro__TX","Parmer__TX","Armstrong__TX","Randall__TX","Deaf_Smith__TX","Carson__TX","Potter__TX","Oldham__TX","Hutchinson__TX","Moore__TX","Hartley__TX","Hansford__TX","Sherman__TX","Dallam__TX","Hudspeth__TX","El_Paso__TX","Lynn__TX","Terry__TX","Yoakum__TX","Dawson__TX","Gaines__TX","Martin__TX","Andrews__TX","Midland__TX","Ector__TX","Crane__TX","Presidio__TX","Culberson__TX","Ward__TX","Winkler__TX","Loving__TX","Reeves__TX","Jeff_Davis__TX","Brewster__TX","Terrell__TX","Pecos__TX","Upton__TX","Reagan__TX","Glasscock__TX","Howard__TX","Borden__TX","Garza__TX","Scurry__TX","Mitchell__TX","Sterling__TX","Irion__TX","Crockett__TX","Val_Verde__TX","Zapata__TX","Jim_Hogg__TX","Starr__TX","Willacy__TX","Hidalgo__TX","Brooks__TX","Kenedy__TX","Kleberg__TX","Jim_Wells__TX","Nueces__TX","San_Patricio__TX","Calhoun__TX","Refugio__TX","Bee__TX","Live_Oak__TX","McMullen__TX","Duval__TX","Webb__TX","La_Salle__TX","Dimmit__TX","Maverick__TX","Kinney__TX","Edwards__TX","Real__TX","Uvalde__TX","Zavala__TX","Frio__TX","Goliad__TX","Victoria__TX","DeWitt__TX","Karnes__TX","Wilson__TX","Atascosa__TX","Medina__TX","Bandera__TX","Kerr__TX","Gillespie__TX","Blanco__TX","Kendall__TX","Bexar__TX","Guadalupe__TX","Comal__TX","Hays__TX","Caldwell__TX","Gonzales__TX","Lavaca__TX","Jackson__TX","Matagorda__TX","Wharton__TX","Colorado__TX","Bastrop__TX","Fayette__TX","Lee__TX","Austin__TX","Washington__TX","Waller__TX","Fort_Bend__TX","Brazoria__TX","Galveston__TX","Montgomery__TX","Harris__TX","Chambers__TX","Hardin__TX","Liberty__TX","Jefferson__TX","Orange__TX","Travis__TX","Kent__TX","Kimble__TX","Sutton__TX","Schleicher__TX","Menard__TX","Mason__TX","Llano__TX","Burnet__TX","Williamson__TX","Burleson__TX","San_Jacinto__TX","Walker__TX","Grimes__TX","Brazos__TX","Madison__TX","Leon__TX","Robertson__TX","Milam__TX","Bell__TX","Coryell__TX","Lampasas__TX","Mills__TX","San_Saba__TX","McCulloch__TX","Concho__TX","Tom_Green__TX","Coke__TX","Runnels__TX","Coleman__TX","Brown__TX","Comanche__TX","Hamilton__TX","Bosque__TX","Falls__TX","McLennan__TX","Hill__TX","Limestone__TX","Freestone__TX","Navarro__TX","Somervell__TX","Ellis__TX","Johnson__TX","Hood__TX","Erath__TX","Eastland__TX","Callahan__TX","Taylor__TX","Nolan__TX","Fisher__TX","Jones__TX","Shackelford__TX","Stephens__TX","Palo_Pinto__TX","Parker__TX","Tarrant__TX","Dallas__TX","Rockwall__TX","Kaufman__TX","Rains__TX","Van_Zandt__TX","Henderson__TX","Anderson__TX","Houston__TX","Trinity__TX","Polk__TX","Newton__TX","Jasper__TX","Tyler__TX","Angelina__TX","Sabine__TX","San_Augustine__TX","Shelby__TX","Nacogdoches__TX","Cherokee__TX","Panola__TX","Rusk__TX","Gregg__TX","Smith__TX","Wood__TX","Harrison__TX","Upshur__TX","Camp__TX","Marion__TX","Cass__TX","Morris__TX","Titus__TX","Franklin__TX","Hopkins__TX","Delta__TX","Hunt__TX","Collin__TX","Denton__TX","Wise__TX","Jack__TX","Young__TX","Throckmorton__TX","Haskell__TX","Stonewall__TX","Dickens__TX","Motley__TX","Hall__TX","Donley__TX","Gray__TX","Roberts__TX","Ochiltree__TX","Lipscomb__TX","Hemphill__TX","Wheeler__TX","Collingsworth__TX","Childress__TX","Cottle__TX","King__TX","Knox__TX","Foard__TX","Hardeman__TX","Archer__TX","Baylor__TX","Wilbarger__TX","Wichita__TX","Clay__TX","Montague__TX","Cooke__TX","Fannin__TX","Grayson__TX","Lamar__TX","Aransas__TX","Cameron__TX","San_Juan__UT","Garfield__UT","Morgan__UT","Davis__UT","Weber__UT","Rich__UT","Cache__UT","Box_Elder__UT","Daggett__UT","Salt_Lake__UT","Summit__UT","Wasatch__UT","Utah__UT","Tooele__UT","Juab__UT","Millard__UT","Sevier__UT","Sanpete__UT","Carbon__UT","Duchesne__UT","Uintah__UT","Grand__UT","Emery__UT","Wayne__UT","Piute__UT","Beaver__UT","Iron__UT","Washington__UT","Kane__UT","Prince_William__VA","Manassas_Park__VA","Accomack__VA","Northampton__VA","Buchanan__VA","Tazewell__VA","Smyth__VA","Bristol__VA","Washington__VA","Russell__VA","Norton__VA","Wise__VA","Dickenson__VA","Scott__VA","Lee__VA","Bland__VA","Grayson__VA","Galax__VA","Wythe__VA","Giles__VA","Radford__VA","Pulaski__VA","Falls_Church__VA","Arlington__VA","Carroll__VA","Henry__VA","Patrick__VA","Floyd__VA","Montgomery__VA","Salem__VA","Franklin_Co___VA","Bedford_Co___VA","Roanoke_Co___VA","Alleghany__VA","Craig__VA","Botetourt__VA","Amherst__VA","Rockbridge__VA","Bath__VA","Highland__VA","Greene__VA","Rockingham__VA","Augusta__VA","Nelson__VA","Buckingham__VA","Appomattox__VA","Campbell__VA","Halifax__VA","Pittsylvania__VA","Martinsville__VA","Danville__VA","Lynchburg__VA","Roanoke__VA","Buena_Vista__VA","Covington__VA","Lexington__VA","Waynesboro__VA","Staunton__VA","Charlottesville__VA","Harrisonburg__VA","Albemarle__VA","Fluvanna__VA","Cumberland__VA","Prince_Edward__VA","Charlotte__VA","Mecklenburg__VA","Colonial_Heights__VA","Lunenburg__VA","Hopewell__VA","Alexandria__VA","Page__VA","Shenandoah__VA","Clarke__VA","Winchester__VA","Frederick__VA","Warren__VA","Rappahannock__VA","Madison__VA","Louisa__VA","Goochland__VA","Orange__VA","Amelia__VA","Powhatan__VA","Nottoway__VA","Brunswick__VA","Dinwiddie__VA","Emporia__VA","Greensville__VA","Poquoson__VA","Franklin__VA","Southampton__VA","Hampton__VA","Williamsburg__VA","Newport_News__VA","Portsmouth__VA","York__VA","James_City__VA","Surry__VA","Sussex__VA","Prince_George__VA","Petersburg__VA","Henrico__VA","Chesterfield__VA","Charles_City__VA","New_Kent__VA","Richmond__VA","Fredericksburg__VA","Fairfax__VA","Manassas__VA","Loudoun__VA","Fauquier__VA","Culpeper__VA","Spotsylvania__VA","Hanover__VA","King_William__VA","King_and_Queen__VA","Middlesex__VA","Gloucester__VA","Lancaster__VA","Richmond_Co___VA","Essex__VA","Caroline__VA","Fairfax_Co___VA","Stafford__VA","King_George__VA","Westmoreland__VA","Northumberland__VA","Mathews__VA","Norfolk__VA","Isle_of_Wight__VA","Suffolk__VA","Chesapeake__VA","Virginia_Beach__VA","Essex__VT","Orleans__VT","Bennington__VT","Grand_Isle__VT","Washington__VT","Caledonia__VT","Lamoille__VT","Franklin__VT","Chittenden__VT","Addison__VT","Rutland__VT","Windham__VT","Windsor__VT","Orange__VT","Okanogan__WA","Whatcom__WA","Wahkiakum__WA","Asotin__WA","Garfield__WA","Columbia__WA","Walla_Walla__WA","Franklin__WA","Whitman__WA","Spokane__WA","Pend_Oreille__WA","Stevens__WA","Lincoln__WA","Adams__WA","Grant__WA","Benton__WA","Klickitat__WA","Clark__WA","Skamania__WA","Cowlitz__WA","Pacific__WA","Lewis__WA","Thurston__WA","Pierce__WA","Yakima__WA","Kittitas__WA","Douglas__WA","Ferry__WA","San_Juan__WA","Island__WA","Snohomish__WA","King__WA","Kitsap__WA","Mason__WA","Grays_Harbor__WA","Jefferson__WA","Clallam__WA","Skagit__WA","Chelan__WA","Door__WI","Kewaunee__WI","Pierce__WI","Pepin__WI","Buffalo__WI","Trempealeau__WI","Jackson__WI","Eau_Claire__WI","Clark__WI","Chippewa__WI","Dunn__WI","St__Croix__WI","Polk__WI","Barron__WI","Rusk__WI","Taylor__WI","Price__WI","Sawyer__WI","Washburn__WI","Burnett__WI","Douglas__WI","Bayfield__WI","Ashland__WI","Iron__WI","Vilas__WI","Oneida__WI","Lincoln__WI","Langlade__WI","Marathon__WI","Wood__WI","Adams__WI","Juneau__WI","Monroe__WI","La_Crosse__WI","Vernon__WI","Sauk__WI","Richland__WI","Crawford__WI","Iowa__WI","Grant__WI","Lafayette__WI","Green__WI","Marquette__WI","Portage__WI","Waupaca__WI","Waushara__WI","Green_Lake__WI","Columbia__WI","Dane__WI","Rock__WI","Kenosha__WI","Racine__WI","Walworth__WI","Dodge__WI","Jefferson__WI","Waukesha__WI","Milwaukee__WI","Ozaukee__WI","Washington__WI","Winnebago__WI","Fond_du_Lac__WI","Sheboygan__WI","Calumet__WI","Manitowoc__WI","Brown__WI","Outagamie__WI","Shawano__WI","Menominee__WI","Oconto__WI","Forest__WI","Florence__WI","Marinette__WI","Berkeley__WV","Jefferson__WV","Cabell__WV","Mingo__WV","Wayne__WV","Wyoming__WV","Logan__WV","McDowell__WV","Mercer__WV","Monroe__WV","Summers__WV","Raleigh__WV","Boone__WV","Lincoln__WV","Putnam__WV","Mason__WV","Jackson__WV","Kanawha__WV","Fayette__WV","Clay__WV","Roane__WV","Calhoun__WV","Wirt__WV","Wood__WV","Pleasants__WV","Ritchie__WV","Gilmer__WV","Braxton__WV","Nicholas__WV","Greenbrier__WV","Webster__WV","Upshur__WV","Lewis__WV","Doddridge__WV","Tyler__WV","Harrison__WV","Hancock__WV","Brooke__WV","Ohio__WV","Marshall__WV","Wetzel__WV","Marion__WV","Barbour__WV","Taylor__WV","Monongalia__WV","Pocahontas__WV","Preston__WV","Randolph__WV","Pendleton__WV","Tucker__WV","Grant__WV","Hardy__WV","Mineral__WV","Hampshire__WV","Morgan__WV","Laramie__WY","Goshen__WY","Teton__WY","Lincoln__WY","Uinta__WY","Sublette__WY","Park__WY","Sweetwater__WY","Fremont__WY","Hot_Springs__WY","Washakie__WY","Big_Horn__WY","Sheridan__WY","Johnson__WY","Natrona__WY","Carbon__WY","Albany__WY","Weston__WY","Crook__WY","Campbell__WY","Converse__WY","Niobrara__WY","Platte__WY","Washington__DC"]
mapchartlist.sort()

truthy = False


In [5]:
# Import and configure population DF for each county

def Make_popdf():
    global popdf
    popdf = pd.DataFrame()
    popdf = pd.read_csv('CountyPopulation.csv', header = None)
    popdf.columns = ['Location', 'Population']
    
    # Convert population to a integer format
    popdf['Population'] = popdf['Population'].str.replace(",", "")
    popdf = popdf.astype({'Population': np.int64})

    # Create separate columns for State and County from "Location"
    popdf['Location'] = popdf['Location'].str.replace(".", "")
    popdf['split'] = popdf['Location'].str.split(",")
    popdf['State'] = popdf['split'].apply(lambda x: str(x[1]).strip())
    popdf['County'] = popdf['split'].apply(lambda x: str(x[0]).strip())
    popdf = popdf.drop('split', 1)
    popdf = popdf.drop('Location', 1)
    popdf['St'] = popdf['State'].apply(lambda x: get_st_abr.get(x))

    # Create Mapchart key county for population DF and check for missed counties
    popdf['MapChart'] = popdf['County'].str.replace(" ", "_") + "__" + popdf['St']

    popdf['MapChart'] = popdf['MapChart'].str.replace("Baltimore_city", "Baltimore_City")        # (Some city & associated counties had difficult
    popdf['MapChart'] = popdf['MapChart'].str.replace("Fairfax_County__VA", "Fairfax_Co___VA")   #  naming conventions to convert without errors.)
    popdf['MapChart'] = popdf['MapChart'].str.replace("Franklin_County__VA", "Franklin_Co___VA")
    popdf['MapChart'] = popdf['MapChart'].str.replace("Richmond_County__VA", "Richmond_Co___VA")
    popdf['MapChart'] = popdf['MapChart'].str.replace("Roanoke_County__VA", "Roanoke_Co___VA")
    popdf['MapChart'] = popdf['MapChart'].str.replace("St_Louis_County__MO", "St_Louis_Co___MO")

    popdf['MapChart'] = popdf['MapChart'].str.replace("_Census_Area", "")
    popdf['MapChart'] = popdf['MapChart'].str.replace("_City_and_Borough", "")
    popdf['MapChart'] = popdf['MapChart'].str.replace("_Borough", "")
    #popdf['MapChart'] = popdf['MapChart'].str.replace("-", "_")
    popdf['MapChart'] = popdf['MapChart'].str.replace("'", "_")
    popdf['MapChart'] = popdf['MapChart'].str.replace("ñ", "n")
    popdf['MapChart'] = popdf['MapChart'].str.replace("St_", "St__")
    popdf['MapChart'] = popdf['MapChart'].str.replace("_County", "")
    popdf['MapChart'] = popdf['MapChart'].str.replace("_Parish", "")
    popdf['MapChart'] = popdf['MapChart'].str.replace("_Municipality", "")

    popdf['MapChart'] = popdf['MapChart'].str.replace("Southeast_Fairbanks", "SE_Fairbanks") # (Some counties had unique naming convention issues
    popdf['MapChart'] = popdf['MapChart'].str.replace("District_of_Columbia", "Washington")  #  that had to be handles separately.)
    popdf['MapChart'] = popdf['MapChart'].str.replace("Bedford__VA", "Bedford_Co___VA")
    popdf['MapChart'] = popdf['MapChart'].str.replace("LaSalle__IL", "La_Salle__IL")
    popdf['MapChart'] = popdf['MapChart'].str.replace("Ste_Genevieve__MO", "Sainte_Genevieve__MO")
    popdf['MapChart'] = popdf['MapChart'].str.replace("St__Mary_s__MD", "St_Mary_s__MD")
    popdf['MapChart'] = popdf['MapChart'].str.replace("Baltimore__MD", "Baltimore_County__MD")

    popdf['MapChart'] = popdf['MapChart'].str.replace("_city", "")

    # Code saved for checking on problems when creating the 'Mapchart' column
    #testmask = popdf['MapChart'].isin(mapchartlist)
    #testdf = popdf[-testmask]
    #testdf.groupby(['State', 'County']).first()

    # Make population DF for States
    global stpopdf
    stpopdf = popdf.groupby(['State']).sum()
    stpopdf = stpopdf.reset_index()
    stpopdf['MapChart'] = stpopdf['State'].apply(lambda x: get_st_abr.get(x))
    stpopdf = stpopdf[['MapChart', 'Population']]
    stpopdf = stpopdf.sort_values(by=['MapChart'])
    stpopdf.to_csv('stpopdf.csv')

    #Get population for all US
    global USpop
    USpop = sum(stpopdf['Population'].to_list())
    
    # Combine NYC County populations
    popdf.at[(popdf.query('MapChart == "New_York__NY"').index[0]), 'Population'] = popdf.query('MapChart == ["Bronx__NY", "Queens__NY", "Kings__NY", "Richmond__NY", "New_York__NY"]').sum()['Population']

    # Final edit and save for population DF
    popdf = popdf[['MapChart', 'Population']]
    popdf = popdf.sort_values(by=['MapChart'])
    popdf.to_csv('popdf.csv')
    main.l_status.configure(text='Population DF 100% Done!')

#Make_popdf()

In [6]:
# Import and configure main DF of Counties

days_to_recovered = 18  # This variable is used to set the estimated mean number of days for people to be considered recovered

def Make_Maindf(Old_DF_Date):
    global df
    def grey_buttons(status):
        main.b_save_dfs.config(state=f"{status}")
        main.b_import_dfs.config(state=f"{status}")
        main.b_pop_dfs.config(state=f"{status}")
        main.b_main_df.config(state=f"{status}")
        main.b_download_master.config(state=f"{status}")
        main.b_update_df.config(state=f"{status}")
        main.b_exit.config(state=f"{status}")
    grey_buttons("disabled")
    def status(place):
        main.l_status.configure(text=f'DFs {place}% Done')
        root.update()
    
    status(0)


    # Initial import and editing
    df2 = pd.read_csv('us-counties.csv', header = 'infer')
    df2 = df2.drop('fips', 1)
    df2 = df2.rename(columns={'date': 'Date', 'county': 'County', 'state': 'State', 'cases': 'Cases', 'deaths': 'Deaths'})
    df2 = df2.astype({'Date': np.datetime64})
    if Old_DF_Date == np.datetime64(df2['Date'].max().date()):
        del df2
        main.l_status.configure(text=f'DFs Already Up to Date')
        grey_buttons("normal")
        return
    if Old_DF_Date != "2020-01-21":
        df2 = df2[(df2['Date'] > Old_DF_Date)]
    df2['St'] = df2['State'].apply(lambda x: get_st_abr.get(x))

    # Create Mapchart key county for main DF and check for missed counties
    df2 = df2.loc[df2['County'] != 'Unknown']
    df2 = df2.loc[df2['State'] != 'Puerto Rico']
    df2 = df2.loc[df2['State'] != 'Northern Mariana Islands']
    df2 = df2.loc[df2['State'] != 'Virgin Islands']
    
    df2['MapChart'] = df2['County'].str.replace(" ", "_") + "__" + df2['St']

    df2['MapChart'] = df2['MapChart'].str.replace("_Census_Area", "")
    df2['MapChart'] = df2['MapChart'].str.replace("_City_and_Borough", "")
    df2['MapChart'] = df2['MapChart'].str.replace("_Borough", "")
    #df2['MapChart'] = d2f['MapChart'].str.replace("-", "_")
    df2['MapChart'] = df2['MapChart'].str.replace("'", "_")
    df2['MapChart'] = df2['MapChart'].str.replace("ñ", "n")
    df2['MapChart'] = df2['MapChart'].str.replace(".", "_")

    df2['MapChart'] = df2['MapChart'].str.replace("Baltimore_city", "Baltimore_City")        # (Some city & associated counties had difficult
    df2['MapChart'] = df2['MapChart'].str.replace("Baltimore__MD", "Baltimore_County__MD")   #  naming conventions to convert without errors.)
    df2['MapChart'] = df2['MapChart'].str.replace("Fairfax__VA", "Fairfax_Co___VA")
    df2['MapChart'] = df2['MapChart'].str.replace("Fairfax_city__VA", "Fairfax__VA")
    df2['MapChart'] = df2['MapChart'].str.replace("Franklin__VA", "Franklin_Co___VA")
    df2['MapChart'] = df2['MapChart'].str.replace("Franklin_city__VA", "Franklin__VA")
    df2['MapChart'] = df2['MapChart'].str.replace("Richmond__VA", "Richmond_Co___VA")
    df2['MapChart'] = df2['MapChart'].str.replace("Richmond_city__VA", "Richmond__VA")
    df2['MapChart'] = df2['MapChart'].str.replace("Roanoke__VA", "Roanoke_Co___VA")
    df2['MapChart'] = df2['MapChart'].str.replace("Roanoke_city__VA", "Roanoke__VA")
    df2['MapChart'] = df2['MapChart'].str.replace("St__Louis__MO", "St__Louis_Co___MO")
    df2['MapChart'] = df2['MapChart'].str.replace("St__Louis_city__MO", "St__Louis__MO")

    df2['MapChart'] = df2['MapChart'].str.replace("New_York_City", "New_York")
    df2['MapChart'] = df2['MapChart'].str.replace("Southeast_Fairbanks", "SE_Fairbanks")    # (Some counties had unique naming convention issues
    df2['MapChart'] = df2['MapChart'].str.replace("District_of_Columbia", "Washington")     #  that had to be handles separately.)
    df2['MapChart'] = df2['MapChart'].str.replace("Bedford__VA", "Bedford_Co___VA")
    df2['MapChart'] = df2['MapChart'].str.replace("LaSalle__IL", "La_Salle__IL")
    df2['MapChart'] = df2['MapChart'].str.replace("Ste__Genevieve__MO", "Sainte_Genevieve__MO")
    df2['MapChart'] = df2['MapChart'].str.replace("St__Mary_s__MD", "St_Mary_s__MD")

    df2['MapChart'] = df2['MapChart'].str.replace("_city", "")
    
    # Code saved for checking on problems when creating the 'Mapchart' column
    #testmask = df2['MapChart'].isin(mapchartlist)
    #testdf = df2[-testmask]
    #testdf.groupby(['State', 'County']).first()
    
    
    # Add Active cases to main DF
    # This finds the number of cases from a previous date for the county and subtracts it from the current number of cases
    # using the 'days_to_recovered' variable. This is a slow process. It is advisable to simply used the 'maindf.csv' file
    # already created unless new csv data has been downloaded from the source website.
    if Old_DF_Date != "2020-01-21":
        start_date1 = Old_DF_Date - np.timedelta64(days_to_recovered, 'D')
        start_date2 = Old_DF_Date - np.timedelta64(7, 'D')
        save_df1 = df[(df['Date'] >= start_date1)]
        save_df2 = df[(df['Date'] >= start_date2)]
        df2 = pd.concat([save_df1, df2])
        del save_df1
    df2 = df2.sort_values(by=['MapChart', 'Date'])
    df2 = df2.reset_index(drop=True)
    df2 = df2.reset_index()
#     DatesDF = df2[(df2['index'] >= 64363) & (df2['index'] <= 64373)]
#     print(DatesDF)

    oldcases = {}
    olddeaths = {}
    counter = 0                                                        # Counter for watching progress
    old_data_dict = {}
    min_date = np.datetime64(df2['Date'].min().date())

    for j, i, a, b, c in zip(df2.MapChart, df2.Date, df2.index, df2.Cases, df2.Deaths):
        if j not in old_data_dict:
            old_data_dict[j] = {}
        old_data_dict[j][i] = [a,b,c]
    for location, dates in old_data_dict.items():
        for date1, list1 in dates.items():
            old_date = date1 - np.timedelta64(days_to_recovered, 'D')
            done = None
            while done != "Yes":
                if old_date in dates:
                    oldcases[list1[0]] = dates[old_date][1]
                    olddeaths[list1[0]] = dates[old_date][2]
                    done = "Yes"
                else:
                    if old_date < min_date:               # if there is no older date entry in the database for earcly dates then old data will be 0
                        oldcases[list1[0]] = 0
                        olddeaths[list1[0]] = 0
                        done = "Yes"
                    else:
                        old_date = old_date - np.timedelta64(1, 'D')
            counter += 1
            if counter % 50000 == 0:
                status(int((counter / (len(df)*2)) * 100))
          
    df2['Active'] = (df2['Cases']- df2['index'].apply(lambda x:oldcases[x])) - (df2['Deaths'] - df2['index'].apply(lambda x: olddeaths[x]))
    df2['Active'] = df2['Active'].apply(lambda x: (abs(x)+x)/2 )         # Change negative values to 0
    df2 = df2.astype({'Active': np.int64})
    if Old_DF_Date != "2020-01-21":
        df2 = df2[(df2['Date'] > Old_DF_Date)]
    status(50)
    

    # Add population data to main DF from population DF
    popdict = dict(zip(popdf.MapChart, popdf.Population))
    df2['Population'] = df2['MapChart'].apply(lambda x: popdict.get(x))
    df2 = df2.dropna(subset=['Population'])
    df2 = df2.astype({'Population': np.int64})


    # Add change in active cases per million people over 1 week
    # Similar process to how the active cases column was created
    df2['Active_Per_Million'] = (df2['Active'] / df2['Population'] * 1000000).round(2)
    oldactive = {}
    counter = 0
    if Old_DF_Date != "2020-01-21":
        df2 = pd.concat([save_df2, df2])
        del save_df2
    df2 = df2.sort_values(by=['MapChart', 'Date'])
    df2 = df2.drop('index', 1)
    df2 = df2.reset_index(drop=True)
    df2 = df2.reset_index()
    
    old_data_dict = {}
    min_date = np.datetime64(df2['Date'].min().date())
    
    for j, i, a, b in zip(df2.MapChart, df2.Date, df2.index, df2.Active):
        if j not in old_data_dict:
            old_data_dict[j] = {}
        old_data_dict[j][i] = [a,b]
    for location, dates in old_data_dict.items():
        for date1, list1 in dates.items():
            old_date = date1 - np.timedelta64(7, 'D')
            done = None
            while done != "Yes":
                if old_date in dates:
                    oldactive[list1[0]] = dates[old_date][1]
                    done = "Yes"
                else:
                    if old_date < min_date:               # if there is no older date entry in the database for earcly dates then old data will be 0
                        oldactive[list1[0]] = 0
                        done = "Yes"
                    else:
                        old_date = old_date - np.timedelta64(1, 'D')
            counter += 1
            if counter % 50000 == 0:
                status(int(((counter / (len(df)*2)) * 100) + 50))

    
    df2['Change_Over_One_Week'] = ((df2['Active']- df2['index'].apply(lambda x: oldactive[x])) / df2['Population'] * 1000000).round(2)
    if Old_DF_Date != "2020-01-21":
        df2 = df2[(df2['Date'] > Old_DF_Date)]
    
    # Final edit, and save for Main DF
    df2['Index'] = df2['Date']
    df2 = df2.set_index('Index')
    df2['Recovered'] = df2['Cases'] - df2['Deaths'] - df2['Active']
    df2['Cases_Per_Million'] = (df2['Cases'] / df2['Population'] * 1000000).round(2)
    df2['Deaths_Per_Million'] = (df2['Deaths'] / df2['Population'] * 1000000).round(2)
    df2['Recovered_Per_Million'] = (df2['Recovered'] / df2['Population'] * 1000000).round(2)
    df2['Mortality_Percent'] = (df2['Deaths'] / df2['Cases'] * 100).round(2)
    df2 = df2[['Date', 'County', 'State', 'Population', 'Cases', 'Deaths', 'Active', 'Recovered', 'Cases_Per_Million', 'Deaths_Per_Million', 'Recovered_Per_Million', 'Active_Per_Million', 'Change_Over_One_Week', 'Mortality_Percent', 'St', 'MapChart']]
    if Old_DF_Date != "2020-01-21":
        df = pd.concat([df, df2])
    else:
        df = df2.copy()
        del df2
    
    df.to_csv('Maindf.csv')
    Make_StateDF()
    Make_US_DF()
    status(100)
    grey_buttons("normal")
    
#Make_Maindf("2020-01-21")

In [7]:
# Create State DF from main DF

def Make_StateDF():
    global StateDF
    StateDF = pd.DataFrame()
    # State data is simply the sum of all subordinate county data
    StateDF = df.groupby(['State', 'Date']).sum()
    StateDF = StateDF.reset_index()
    StateDF['St'] = StateDF['State'].apply(lambda x: get_st_abr[x])
    # Mapchart column for states is easy because mapchart.net just uses the abbreviation as a handle
    StateDF['MapChart'] = StateDF['State'].apply(lambda x: get_st_abr[x])
    # Add state population data to the DF
    popdict = dict(zip(stpopdf.MapChart, stpopdf.Population))
    StateDF['Population'] = StateDF['MapChart'].apply(lambda x: popdict[x])
    #StateDF = StateDF.dropna(subset=['Population'])
    StateDF = StateDF.astype({'Population': np.int64})
    
    # Columns with ratios need to be recalculated because sum on individual counties is bad math
    StateDF['Cases_Per_Million'] = (StateDF['Cases'] / StateDF['Population'] * 1000000).round(2)
    StateDF['Deaths_Per_Million'] = (StateDF['Deaths'] / StateDF['Population'] * 1000000).round(2)
    StateDF['Recovered_Per_Million'] = (StateDF['Recovered'] / StateDF['Population'] * 1000000).round(2)
    StateDF['Active_Per_Million'] = (StateDF['Active'] / StateDF['Population'] * 1000000).round(2)
    StateDF['Mortality_Percent'] = (StateDF['Deaths'] / StateDF['Cases'] * 100).round(2)
    
    # Add change in cases per million people over 1 week
    # Same process done for main DF
    StateDF = StateDF.sort_values(by=['MapChart','Date'])
    StateDF = StateDF.reset_index(drop=True)
    StateDF = StateDF.reset_index()

    oldactive = {}
    old_data_dict = {}
    for j, i, a, b in zip(StateDF.MapChart, StateDF.Date, StateDF.index, StateDF.Active):
        if j not in old_data_dict:
            old_data_dict[j] = {}
        old_data_dict[j][i] = [a,b]
    for location, dates in old_data_dict.items():
        for date1, list1 in dates.items():
            old_date = date1 - np.timedelta64(7, 'D')
            done = None
            while done != "Yes":
                if old_date in dates:
                    oldactive[list1[0]] = dates[old_date][1]
                    done = "Yes"
                else:
                    if str(old_date) < "2020-01-20":               # if there is no older date entry in the database for earcly dates then old data will be 0
                        oldactive[list1[0]] = 0
                        done = "Yes"
                    else:
                        old_date = old_date - np.timedelta64(1, 'D')

    StateDF['Change_Over_One_Week'] = ((StateDF['Active']- StateDF['index'].apply(lambda x: oldactive[x])) / StateDF['Population'] * 1000000).round(2)
    
    # Final edit, and save for State DF  
    StateDF['County'] = StateDF['St']
    StateDF['Index'] = StateDF['Date']
    StateDF = StateDF.set_index('Index')
    StateDF = StateDF.drop('index', 1)
    StateDF = StateDF[['Date', 'County', 'State', 'Population', 'Cases', 'Deaths', 'Active', 'Recovered', 'Cases_Per_Million', 'Deaths_Per_Million', 'Recovered_Per_Million', 'Active_Per_Million', 'Change_Over_One_Week', 'Mortality_Percent', 'St', 'MapChart']]
    StateDF.to_csv('StateDF.csv')
    
#Make_StateDF()

In [8]:
# Create US DF from state DF

def Make_US_DF():
    global US_DF
    US_DF = pd.DataFrame()
    # US data is simply the sum of all subordinate State data
    US_DF = StateDF.groupby(['Date']).sum()
    US_DF = US_DF.reset_index()
    US_DF['State'] = "USA"
    US_DF['County'] = "USA"
    US_DF['St'] = "USA"
    US_DF['MapChart'] = "USA"
    # Add population data to the DF
    US_DF['Population'] = USpop
    US_DF = US_DF.astype({'Population': np.int64})
    
    # Columns with ratios need to be recalculated because sum on individual states is bad math
    US_DF['Cases_Per_Million'] = (US_DF['Cases'] / US_DF['Population'] * 1000000).round(2)
    US_DF['Deaths_Per_Million'] = (US_DF['Deaths'] / US_DF['Population'] * 1000000).round(2)
    US_DF['Recovered_Per_Million'] = (US_DF['Recovered'] / US_DF['Population'] * 1000000).round(2)
    US_DF['Active_Per_Million'] = (US_DF['Active'] / US_DF['Population'] * 1000000).round(2)
    US_DF['Mortality_Percent'] = (US_DF['Deaths'] / US_DF['Cases'] * 100).round(2)
    
    # Add change in cases per million people over 1 week
    # Same process done for main DF
    US_DF = US_DF.sort_values(by=['Date'])
    US_DF = US_DF.reset_index(drop=True)
    US_DF = US_DF.reset_index()

    oldactive = {}
    old_data_dict = {}
    for i, a, b in zip(US_DF.Date, US_DF.index, US_DF.Active):
        old_data_dict[i] = [a,b]
    for date1, list1 in old_data_dict.items():
        old_date = date1 - np.timedelta64(7, 'D')
        done = None
        while done != "Yes":
            if old_date in old_data_dict:
                oldactive[list1[0]] = old_data_dict[old_date][1]
                done = "Yes"
            else:
                if str(old_date) < "2020-01-20":               # if there is no older date entry in the database for earcly dates then old data will be 0
                    oldactive[list1[0]] = 0
                    done = "Yes"
                else:
                    old_date = old_date - np.timedelta64(1, 'D')

    US_DF['Change_Over_One_Week'] = ((US_DF['Active']- US_DF['index'].apply(lambda x: oldactive[x])) / US_DF['Population'] * 1000000).round(2)
    
    # Final edit, and save for State DF  
    US_DF['Index'] = US_DF['Date']
    US_DF = US_DF.set_index('Index')
    US_DF = US_DF.drop('index', 1)
    US_DF = US_DF[['Date', 'County', 'State', 'Population', 'Cases', 'Deaths', 'Active', 'Recovered', 'Cases_Per_Million', 'Deaths_Per_Million', 'Recovered_Per_Million', 'Active_Per_Million', 'Change_Over_One_Week', 'Mortality_Percent', 'St', 'MapChart']]
    US_DF.to_csv('US_DF.csv')
    main.recent = np.datetime64(df['Date'].max().date())
    main.l_dates.configure(text=f"""
        Available Date Ranges: (2020-01-21 : {main.recent})""", font=("Arial Bold",15))

#Make_US_DF()

In [9]:
# Import all DFs from csv Files; just an option in the ADMIN menu

def Import_DF():
    global popdf
    popdf = pd.read_csv('popdf.csv', header = 'infer')
    global df
    df = pd.read_csv('Maindf.csv', header = 'infer')
    df = df.set_index('Index')
    df = df.astype({'Date': np.datetime64})
    global StateDF
    StateDF = pd.read_csv('StateDF.csv', header = 'infer')
    StateDF = StateDF.set_index('Index')
    StateDF = StateDF.astype({'Date': np.datetime64})
    main.l_status.configure(text='Import DFs 100% Done!')

#Import_DF()

In [10]:
# Save all DFs to csv Files; just an option in the ADMIN menu

def Save_DF():
    popdf.to_csv('popdf.csv')
    df.to_csv('Maindf.csv')
    StateDF.to_csv('StateDF.csv')
    US_DF.to_csv('US_DF.csv')
    main.l_status.configure(text='Save DFs 100% Done!')
    
#Save_DF()

In [11]:
# Maps function; creates a string of code that can be copy/pasted at mapcharts.net to easily generate a map of the data

def maps(map1, biglabel, date, place):
    # Determine which variables are used as the map key depending on map type
    if map1 in [1,2,3,4]:
        l = [1000000, 100000, 10000, 1000, 100, 10, 0]
        h = [10000000, 1000000, 100000, 10000, 1000, 100, 10]
        labels = ['> 1M','100k - 1M','10k - 100k','1k - 10k','100 - 1000','10 - 100','< 10','0']
    elif map1 in [5,6, 7, 8]:
        l = [100000, 10000, 1000, 100, 10, 1, 0]
        h = [1000000, 100000, 10000, 1000, 100, 10, 1]
        labels = ['> 100k','10k - 100k','1k - 10k','100 - 1000','10 - 100','1 - 10','< 1','0']
    elif map1 == 9:
        l = [-1000000,-1000,-100,-10,1,10,100,1000]
        h = [-1000,-100,-10,-1,10,100,1000, 1000000]
        labels = ["> (-1k) Decrease","(-100) - (-1000) Decrease","(-10) - (-100) Decrease","(-1) - (-10) Decrease","(-1) - (+1) Little Change",
                  "(+1) - (+10) Increase","(+10) - (+100) Increase","(+100) - (+1000) Increase","> (+1k) Increase"]
    elif map1 == 10:
        l = [15, 10, 8, 6, 4, 2, 0]
        h = [100, 15, 10, 8, 6, 4, 2]
        labels = ['> 15%','10% - 15%','8% - 10%','6% - 8%','4% - 6%','2% - 4%','< 2%','Less than 1000 Cases Reported']

    # Determine map color codes based on map type and generate lists of lists for each to store locations in
    if map1 == 9:
        colors = ['#004529','#238443','#78c679','#d9f0a3','#d9d9d9',"#fbb4ae","#ff2000","#d7301f","#7f0000"]
        grey, llgreen, lgreen, green, dgreen, llred, lred, red, dred = ([] for i in range(9))
        grey_dict, llgreen_dict, lgreen_dict, green_dict, dgreen_dict, llred_dict, lred_dict, red_dict, dred_dict = ({} for i in range(9))
        biglist = [dgreen, green, lgreen, llgreen, llred, lred, red, dred]
        bigdictlist = [dgreen_dict, green_dict, lgreen_dict, llgreen_dict, grey_dict, llred_dict, lred_dict, red_dict, dred_dict]
    else:
        colors = ['#810f7c','#8c6bb1','#b30000','#ff2000','#ff7f00','#ffff33','#41ab5d','#d9d9d9']
        grey, green, yellow, orange, red, maroon, lpurple, dpurple = ([] for i in range(8))
        grey_dict, green_dict, yellow_dict, orange_dict, red_dict, maroon_dict, lpurple_dict, dpurple_dict = ({} for i in range(8))
        biglist = [dpurple, lpurple, maroon, red, orange, yellow, green]
        bigdictlist = [dpurple_dict, lpurple_dict, maroon_dict, red_dict, orange_dict, yellow_dict, green_dict, grey_dict]

    # Create a DF of all the data for the date selected
    if place == 'State':
        datadf = StateDF[StateDF['Date'] == date]
    elif place == 'County':
        datadf = df[df['Date'] == date]
        
    # Create a dictionary of data from the column associated with the desired map type
    if map1 == 1: datadict = dict(zip(datadf.MapChart, datadf.Cases))
    elif map1 == 2: datadict = dict(zip(datadf.MapChart, datadf.Deaths))
    elif map1 == 3: datadict = dict(zip(datadf.MapChart, datadf.Recovered))
    elif map1 == 4: datadict = dict(zip(datadf.MapChart, datadf.Active))
    elif map1 == 5: datadict = dict(zip(datadf.MapChart, datadf.Cases_Per_Million))
    elif map1 == 6: datadict = dict(zip(datadf.MapChart, datadf.Deaths_Per_Million))
    elif map1 == 7: datadict = dict(zip(datadf.MapChart, datadf.Recovered_Per_Million))
    elif map1 == 8: datadict = dict(zip(datadf.MapChart, datadf.Active_Per_Million))
    elif map1 == 9: datadict = dict(zip(datadf.MapChart, datadf.Change_Over_One_Week))
    # The mortality rate map only requires a minimum of 1000 cases to make it statistically significant
    elif map1 == 10:
        greydf = datadf[datadf['Cases'] < 1000]
        datadf = datadf[datadf['Cases'] >= 1000]
        greylist = greydf['MapChart'].to_list()
        for location in greylist:
            grey.append(location)
        datadict = dict(zip(datadf.MapChart, datadf.Mortality_Percent))
    # Determine which color code list each county belongs in for the map
    for k, v in datadict.items():
        if v == 0:
            grey.append(k)
        else:
            for i in range(len(biglist)):
                if v != 0 and l[i] <= v < h[i]:
                    biglist[i].append(k)
                    if k == "New_York__NY":
                        biglist[i].extend(["Bronx__NY", "Queens__NY", "Kings__NY", "Richmond__NY"])
                                                    # NY, NY does not have unique data for each county so all will be colored the same
                       
    # Add 0 value (grey) counties to the color lists
    if map1 == 9:
        biglist.insert(4, grey)
    else:
        biglist.append(grey)
    for i, list1 in enumerate(biglist):
        if len(list1) != 0:
            bigdictlist[i] = {"div":f"#box{i}","label":f"{labels[i]}","paths":list1}
    all_colors = {k: v for k, v in zip(colors, bigdictlist)}
    # Remove all empty color code lists to prevent errors on the mapchart.net website
    for key, value in list(all_colors.items()):
        if len(value) == 0:
            del(all_colors[key])
    # Generate string value of the final code format needed for upload at mapchart.net
    result = {"groups":all_colors,"title":f'{biglabel}{date}',"hidden":[],"background":"#1f78b4","borders":"#000000"}
    return str(result).replace("'",'"')


In [12]:
# Menu for graphs

def Graphs(start, end, locations, data, type1):
    days = mdates.DayLocator()
    week = mdates.WeekdayLocator()
    
    Answer = Slice_Database(start, end, locations, data)
    Answer = Answer.sort_values(by=['Date'])
    loc_menu4 = Toplevel()
    #loc_menu3.geometry('600x400+200+100')
    loc_menu4.title('Table app')
    f = Frame(loc_menu4)
    f.pack(fill=BOTH,expand=1)


    def datalabels(data1):
        if data1 == 1:
            data = "Cases"
            title = "Total Cumulative Cases"
        elif data1 == 2:
            data = "Deaths"
            title = "Total Deaths"
        elif data1 == 3:
            data = "Recovered"
            title = "Total Recoveries"
        elif data1 == 4:
            data = "Active"
            title = "Total Active Cases"
        elif data1 == 5:
            data = "Cases_Per_Million"
            title = "Cumulative Cases per Million People"
        elif data1 == 6:
            data = "Deaths_Per_Million"
            title = "Deaths per Million People"
        elif data1 == 7:
            data = "Recovered_Per_Million"
            title = "Recoveries per Million People"
        elif data1 == 8:
            data = "Active_Per_Million"
            title = "Active Cases per Million People"
        elif data1 == 9:
            data = "Change_Over_One_Week"
            title = "Change in Active Cases per Million People over 1 Week"
        elif data1 == 10:
            data = "Mortality_Percent"
            title = "% Mortality Rates"
        return data, title

    
    fig, ax = plt.subplots()
    
    def get_color(d):
        if d == 1 or d == 5 or d == 9: return 'black'
        if d == 2 or d == 6 or d == 10: return 'red'
        if d == 3 or d == 7: return 'green'
        if d == 4 or d == 8: return 'orange'
        
    
    
    if type1  == 'one_location':
        for k, v in locations.items():
            title = f"{v[0]}"
            for d in data:
                data1, title1 = datalabels(d)
                x = Answer['Date']
                y = Answer[data1]
                plt.plot(x, y, c = get_color(d), label = title1)
            if data[0] in [1,2,3,4]:
                plt.ylabel('Totals')
            elif data[0] in [5,6,7,8]:
                plt.ylabel('Per Million People')
            else:
                plt.ylabel(title1)
            
    elif type1  == 'many_locations':
        data1, title1 = datalabels(data[0])
        title = f"{title1}"
        plt.ylabel(title1)
        

        for k, v in locations.items():
            tempdf = Answer.query(f'MapChart == "{k}"')
            x = tempdf['Date']
            y = tempdf[data1]
            plt.plot(x, y, c = np.random.rand(3), label = v[0])


    plt.xlabel('Date')
    plt.title(title)
    plt.legend()
    plt.grid(True, color = "#dddddd")
    fig.set_size_inches(14, 10, forward=True)
    fig.autofmt_xdate(bottom=0.2, rotation=30, ha='right')
    ax.xaxis.set_minor_locator(days)
    ax.xaxis.set_major_locator(week)
    fig.savefig('foo.png')
    icon = PhotoImage(file = "foo.png")
    lbl1 = Label(f, image = icon)
    lbl1.image = icon
    lbl1.pack()
    plt.close()
    

In [13]:
# Spreadsheet
def Spreadsheet(start, end, locations, data):
    xdf = Slice_Database(start, end, locations, data)
    loc_menu3 = Toplevel()
    #loc_menu3.geometry('600x400+200+100')
    loc_menu3.title('Table app')
    f = Frame(loc_menu3)
    f.pack(fill=BOTH,expand=1)
    table = pt = Table(f, dataframe=xdf,showtoolbar=True, showstatusbar=True)
    pt.show()
    

In [14]:
# Getting specific dates and locations from DFs

def Slice_Database(start, end, locations, data):
    USAdict = {}
    Statesdict = {}
    Countiesdict = {}
    UserDF = pd.DataFrame()
    for k, v in locations.items():
        if v[1] == 'USA':
            USAdict[k] = v
        elif v[1] == 'State' or v[1] == 'All_States':
            Statesdict[k] = v
        elif v[1] == 'County' or v[1] == 'All_Counties' or v[1] == 'All_St_Counties':
            Countiesdict[k] = v

    if len(USAdict) > 0:
        DatesDF = US_DF[(US_DF['Date'] >= start) & (US_DF['Date'] <= end)]
        UserDF = UserDF.append(DatesDF)
    if len(Statesdict) > 0:
        DatesDF = StateDF[(StateDF['Date'] >= start) & (StateDF['Date'] <= end)]
        for k, v in Statesdict.items():
            if v[1] == 'All_States':
                UserDF = UserDF.append(DatesDF)
            else:
                LocDF = DatesDF.query(f'St == "{k}"')
                UserDF = UserDF.append(LocDF)
    if len(Countiesdict) > 0:
        DatesDF = df[(df['Date'] >= start) & (df['Date'] <= end)]
        for k, v in Countiesdict.items():
            if v[1] == 'All_Counties':
                UserDF = UserDF.append(DatesDF)
            elif v[1] == 'All_St_Counties':
                LocDF = DatesDF.query(f'State == "{k[4:-15]}"')
                UserDF = UserDF.append(LocDF)
            else:
                LocDF = DatesDF.query(f'MapChart == "{k}"')
                UserDF = UserDF.append(LocDF)
    
    
    if 1 not in data: UserDF = UserDF.drop('Cases', 1)
    if 2 not in data: UserDF = UserDF.drop('Deaths', 1)
    if 3 not in data: UserDF = UserDF.drop('Recovered', 1)
    if 4 not in data: UserDF = UserDF.drop('Active', 1)
    if 5 not in data: UserDF = UserDF.drop('Cases_Per_Million', 1)
    if 6 not in data: UserDF = UserDF.drop('Deaths_Per_Million', 1)
    if 7 not in data: UserDF = UserDF.drop('Recovered_Per_Million', 1)
    if 8 not in data: UserDF = UserDF.drop('Active_Per_Million', 1)
    if 9 not in data: UserDF = UserDF.drop('Change_Over_One_Week', 1)
    if 10 not in data: UserDF = UserDF.drop('Mortality_Percent', 1)
        
    return UserDF


In [15]:
def Map_Export_Menu(type1, date, place):
    if type1 == 1: data = (maps(1, f"Total Cumulative Covid-19 Cases ", date, place))
    elif type1 == 2: data = (maps(2, f"Total Covid-19 Deaths ", date, place))
    elif type1 == 3: data = (maps(3, f"Total Covid-19 Cases Recovered ", date, place))
    elif type1 == 4: data = (maps(4, f"Total Active Covid-19 Cases ", date, place))
    elif type1 == 5: data = (maps(5, f"Covid-19 Cumulative Cases Per Million People ", date, place))
    elif type1 == 6: data = (maps(6, f"Covid-19 Deaths Per Million People ", date, place))
    elif type1 == 7: data = (maps(7, f"Covid-19 Recoveries Per Million People ", date, place))
    elif type1 == 8: data = (maps(8, f"Active Covid-19 Cases Per Million People ", date, place))
    elif type1 == 9: data = (maps(9, f"Change in Covid-19 Cases per Million People over 1 Week ", date, place))
    elif type1 == 10: data = (maps(10, f"Covid-19 Mortality Rates ", date, place))
    
    
    if place == 'State': website = "https://mapchart.net/usa.html"
    if place  == 'County': website = "https://mapchart.net/usa-counties.html"

    
    data_menu = Toplevel()
    data_menu.title("MapChart.net Data")
    title_frame = Frame(data_menu, relief=RAISED, borderwidth=1)
    title_frame.pack(fill='x')
    data_frame = Frame(data_menu, relief=RAISED, borderwidth=1)
    data_frame.pack(fill='x')
    button_frame = Frame(data_menu, relief=RAISED, borderwidth=1)
    button_frame.pack(fill='x')
    def copyit():
        data_menu.withdraw()
        data_menu.clipboard_clear()
        data_menu.clipboard_append(data)
        data_menu.update()
    def copyit2():
        data_menu.withdraw()
        data_menu.clipboard_clear()
        data_menu.clipboard_append(data)
        data_menu.update()
        webbrowser.open_new(website)
        data_menu.destroy()
    
    l_title = Label(title_frame, text = f"""Copy the following data.
    Then go to the following website:
    {website}
    Paste the below data into the 
    "Save - Upload Map Configuration" Menu""")
    l_title.pack(padx=5, pady=5)
    my_txt = StringVar()
    my_txt.set(data)
    my_msg = Entry(data_frame, text=my_txt, justify='center', state='readonly')
    my_msg.pack(expand=True, fill=BOTH)
    b_copyit = Button(button_frame, text = "Copy to Clipboard", command = copyit)
    b_copyit.pack(side='left', padx=15, pady=15)
    b_copyit = Button(button_frame, text = "Copy Data and Open Website", command = copyit2)
    b_copyit.pack(side='right', padx=15, pady=15)


In [16]:
# Select locations menus

def clear_locations(self):
    self.locations.clear()
    self.l_locations.configure(text=f"{self.locations_disp()}", wraplength=700)
    
    
def select_usa(self):
    if "USA" not in self.locations:
        self.locations['USA'] = (['USA', 'USA'])
    elif "USA" in self.locations:
        del self.locations['USA']
    self.l_locations.configure(text=f"{self.locations_disp()}", wraplength=700)

    
def select_states(self):
    dict1 = get_st_abr.copy()
    loc_menu = Toplevel(self)
    loc_menu.title("Locations")
    button_frame = Frame(loc_menu, relief=RAISED, borderwidth=1)
    button_frame.pack(fill='x')
    boxes_frame = Frame(loc_menu, relief=RAISED, borderwidth=1)
    boxes_frame.pack(fill='x')
    col = 0
    row = 0
    for k, v in get_st_abr.items():
        dict1[k] = BooleanVar()
        dict1[k].set(False)
        if v in self.locations:
            dict1[k].set(True)
        c = Checkbutton(boxes_frame, text=f'{k}', variable=dict1[k])
        if row == 9:
            row = 0
            col += 1
        c.grid(sticky=W, column = col, row = row)
        row += 1
    
    def all_states():
        global truthy
        truthy = not truthy
        for k, v in get_st_abr.items():
            dict1[k].set(truthy)
    def save():
        for k, v in dict1.items():
            if v.get() == 1 and get_st_abr[k] not in self.locations:
                self.locations[get_st_abr[k]] = ([k, 'State'])
            elif v.get() == 0 and get_st_abr[k] in self.locations:
                del self.locations[get_st_abr[k]]

        self.l_locations.configure(text=f"{self.locations_disp()}", wraplength=700)
        loc_menu.destroy()
    l_title = Label(button_frame, text="All of Each State as One Data Set", font=("Arial Bold",15))
    l_title.pack(padx=5, pady=5)
    b_save = Button(button_frame, text = "Save Choices", command = save)
    b_save.pack(side='left', padx=100, pady=5)
    b_all_states = Button(button_frame, text = "Select/Unselect All States", command = all_states)
    b_all_states.pack(side='right', padx=100, pady=5)

def select_counties(self):
    loc_menu = Toplevel(self)
    loc_menu.title("Locations")
    button_frame = Frame(loc_menu, relief=RAISED, borderwidth=1)
    button_frame.pack(fill='x')
    boxes_frame = Frame(loc_menu, relief=RAISED, borderwidth=1)
    boxes_frame.pack(fill='x')
    col = 0
    row = 0
    r_state = StringVar()
    for k, v in get_st_abr.items():
        r = Radiobutton(boxes_frame, text=f'{k}', value = v, variable = r_state, tristatevalue="x")
        if row == 9:
            row = 0
            col += 1
        r.grid(sticky=W, column = col, row = row)
        row += 1
    def see_counties():
        select_counties_sub(self, r_state.get())
    def all_counties():
        countiesdict1 = dict(zip(df.MapChart, df.County))
        test = self.locations_disp()
        if 'All Counties Added' in test:
            for k, v in countiesdict1.items():
                if k in self.locations:
                    del self.locations[k]
        elif 'All Counties Added' not in test:
            for k, v in countiesdict1.items():
                self.locations[k] = ([f'{countiesdict1[k]}, {str(k)[-2:]}', 'County'])

        self.l_locations.configure(text=f"{self.locations_disp()}", wraplength=700)
    l_title = Label(button_frame, text="Select a State to See it's Counties", font=("Arial Bold",15))
    l_title.pack(padx=5, pady=5)
    b_save = Button(button_frame, text = "See Counties", command = see_counties)
    b_save.pack(side='left', padx=50, pady=5)
    b_done = Button(button_frame, text = "Done", command = lambda : loc_menu.destroy())
    b_done.pack(side='right', padx=50, pady=5)
    b_all_states = Button(button_frame, text = "Select/Unselect All US Counties", command = all_counties)
    b_all_states.pack(side='right', padx=50, pady=5)

    
def select_counties_sub(self, st):
    if st == "":
        return
    tempdf = df.query(f'(St == "{st}") & (Date == "{np.datetime64(df["Date"].max().date())}")') # Generate list of counties for selected state
    countiesdict1 = dict(zip(tempdf.County, tempdf.MapChart))
    countiesdict2 = countiesdict1.copy()
    
    loc_menu2 = Toplevel(self)
    loc_menu2.title(f"{get_st_full[st]} Counties")
    button_frame = Frame(loc_menu2, relief=RAISED, borderwidth=1)
    button_frame.pack(fill='x')
    boxes_frame = Frame(loc_menu2, relief=RAISED, borderwidth=1)
    boxes_frame.pack(fill='x')
    col = 0
    row = 0
    for k, v in countiesdict1.items():
        countiesdict2[k] = BooleanVar()
        countiesdict2[k].set(False)
        if v in self.locations or str(f"All {get_st_full[st]} Counties Added") in self.locations:
            countiesdict2[k].set(True)
        c = Checkbutton(boxes_frame, text=f'{k}', variable=countiesdict2[k])
        if row == 19:
            row = 0
            col += 1
        c.grid(sticky=W, column = col, row = row)
        row += 1

    def all_counties():
        global truthy
        truthy = not truthy
        for k, v in countiesdict2.items():
            countiesdict2[k].set(truthy)
    def save():
        for k, v in countiesdict2.items():
            if v.get() == 1 and countiesdict1[k] not in self.locations:
                self.locations[countiesdict1[k]] = ([f'{k}, {st}', 'County'])
            elif v.get() == 0 and countiesdict1[k] in self.locations:
                del self.locations[countiesdict1[k]]
    
        self.l_locations.configure(text=f"{self.locations_disp()}", wraplength=700)
        loc_menu2.destroy()
    l_title = Label(button_frame, text=f"{get_st_full[st]} Counties", font=("Arial Bold",15))
    l_title.pack(padx=5, pady=5)
    b_save = Button(button_frame, text = "Save Choices", command = save)
    b_save.pack(side='left', padx=100, pady=5)
    b_all_counties = Button(button_frame, text = "Select/Unselect All Counties", command = all_counties)
    b_all_counties.pack(side='right', padx=100, pady=5)


In [17]:
# Admin Menu

def admin(self):
    def exit_admin():
        admin_menu.destroy()
    self.status_disp = "status window"
    admin_menu = Toplevel(self)
    admin_menu.title("Admin Menu")
    display_frame = Frame(admin_menu, relief=RAISED, borderwidth=1)
    display_frame.pack(fill='x')
    button_frame = Frame(admin_menu, relief=RAISED, borderwidth=1)
    button_frame.pack(fill='both')
    l_title = Label(display_frame, text="Admin Menu", font=("Arial Bold",15))
    l_title.pack(padx=5, pady=5)
    self.l_status = Label(display_frame, text=f"{self.status_disp}")
    self.l_status.pack(padx=5, pady=5)
    self.b_save_dfs = Button(button_frame, text = "Save DFs", command = Save_DF)
    self.b_save_dfs.pack(padx=5, pady=5)
    self.b_import_dfs = Button(button_frame, text = "Import Saved DFs", command = Import_DF)
    self.b_import_dfs.pack(padx=5, pady=5)
    self.b_pop_dfs = Button(button_frame, text = "Make Population DF", command = Make_popdf)
    self.b_pop_dfs.pack(padx=5, pady=5)
    self.b_download_master = Button(button_frame, text = "Download New Master csv File", command = self.download_master)
    self.b_download_master.pack(padx=5, pady=5)
    self.b_main_df = Button(button_frame, text = "Make New DFs", command = lambda : Make_Maindf("2020-01-21"))
    self.b_main_df.pack(padx=5, pady=5)
    self.b_update_df = Button(button_frame, text = "Update All DFs", command = lambda : Make_Maindf(np.datetime64(df['Date'].max().date())))
    self.b_update_df.pack(padx=5, pady=5)
    self.b_exit = Button(button_frame, text = "Exit", command = exit_admin)
    self.b_exit.pack(padx=5, pady=5)


In [ ]:
class MainWindow(Frame):

    def __init__(self, *args, **kwargs):
        Frame.__init__(self, *args, **kwargs)
        self.recent = np.datetime64(df['Date'].max().date())
        self.start = np.datetime64(df['Date'].min().date())
        self.end = np.datetime64(df['Date'].max().date())
        self.locations = {}

        self.c_total_v = BooleanVar()
        self.c_total_v.set(False)
        self.c_deaths_v = BooleanVar()
        self.c_deaths_v.set(False)
        self.c_recover_v = BooleanVar()
        self.c_recover_v.set(False)
        self.c_active_v = BooleanVar()
        self.c_active_v.set(False)
        self.c_total_permil_v = BooleanVar()
        self.c_total_permil_v.set(False)
        self.c_deaths_permil_v = BooleanVar()
        self.c_deaths_permil_v.set(False)
        self.c_recover_permil_v = BooleanVar()
        self.c_recover_permil_v.set(False)
        self.c_active_permil_v = BooleanVar()
        self.c_active_permil_v.set(False)
        self.c_change_permil_v = BooleanVar()
        self.c_change_permil_v.set(False)
        self.c_mortality_v = BooleanVar()
        self.c_mortality_v.set(False)

        self.mapchartdict = {}
        temp_mapchartlist = df['MapChart'].to_list()
        temp_mapchartlist = list(set(temp_mapchartlist))
        for location in temp_mapchartlist:
            st = location[-2:]
            if st not in self.mapchartdict:
                self.mapchartdict[st] = []
            self.mapchartdict[st].append(location)
        
        root.title("Covid-19 Database")
#        root.geometry('600x600')
        dates_frame = Frame(self, relief=RAISED, borderwidth=1)
        dates_frame.pack(fill='x')
        actions_frame = Frame(self, relief=RAISED, borderwidth=1)
        actions_frame.pack(fill='x')        
        start_frame = Frame(self, relief=RAISED, borderwidth=1)
        start_frame.pack(fill='x')
        end_frame = Frame(self, relief=RAISED, borderwidth=1)
        end_frame.pack(fill='x')
        data1_frame = Frame(self, relief=RAISED, borderwidth=1)
        data1_frame.pack(fill='x')
        data2_frame = Frame(self, relief=RAISED, borderwidth=1)
        data2_frame.pack(fill='x')
        data3_frame = Frame(data2_frame, relief=RAISED, borderwidth=1)
        data3_frame.pack(side='left')
        data4_frame = Frame(data2_frame, relief=RAISED, borderwidth=1)
        data4_frame.pack(side='left')
        data5_frame = Frame(data2_frame, relief=RAISED, borderwidth=1)
        data5_frame.pack(side='left')
        data6_frame = Frame(data5_frame, relief=RAISED, borderwidth=1)
        data6_frame.pack(fill='x')
        data7_frame = Frame(data5_frame, relief=RAISED, borderwidth=1)
        data7_frame.pack(fill='x')
        location1_frame = Frame(self, relief=RAISED, borderwidth=1)
        location1_frame.pack(fill='x')
        location2_frame = Frame(location1_frame, relief=RAISED, borderwidth=1)
        location2_frame.pack(fill='x')
        location3_frame = Frame(location1_frame, relief=RAISED, borderwidth=1)
        location3_frame.pack(fill='x')
        location4_frame = Frame(location1_frame, relief=RAISED, borderwidth=1)
        location4_frame.pack(fill='x')   



        self.l_dates = Label(dates_frame, text=f"""
        Available Date Ranges: (2020-01-21 : {self.recent})""", font=("Arial Bold",15))
        self.l_dates.pack()
        b_admin = Button(dates_frame, text = "Admin", command = lambda : admin(self))
        b_admin.pack(side='left', padx=5, pady=5)
        b_master_exit = Button(dates_frame, text = "EXIT", command = self.master_exit)
        b_master_exit.pack(side='right', padx=5, pady=5)     
        
        b_state_map = Button(actions_frame, text = "State Map", command = lambda : self.check_data('state_map'))
        b_state_map.pack(side='left', padx=15, pady=5)
        b_county_map = Button(actions_frame, text = "County Map", command = lambda : self.check_data('county_map'))
        b_county_map.pack(side='left', padx=15, pady=5)
        b_location_graph = Button(actions_frame, text = "Graph one Location", command = lambda : self.check_data('one_location'))
        b_location_graph.pack(side='left', padx=15, pady=5)
        b_data_graph = Button(actions_frame, text = "Graph Multiple Locations", command = lambda : self.check_data('many_locations'))
        b_data_graph.pack(side='left', padx=15, pady=5)
        b_spreadsheet = Button(actions_frame, text = "See in Spreadsheet", command = lambda : self.check_data('spreadsheet'))
        b_spreadsheet.pack(side='left', padx=15, pady=5)  
        
        
        def all_dates():
            self.start = np.datetime64("2020-01-21")
            l_start.configure(text = f"Start Date: {self.start}", font=("Arial Bold",15))
            self.end = self.recent
            l_end.configure(text = f"End Date:  {self.end}", font=("Arial Bold",15))
        def set_start():
            self.start = np.datetime64(e_start.get())
            l_start.configure(text = f"Start Date: {self.start}", font=("Arial Bold",15))
        b_start = Button(start_frame, text = "Set Date", command = set_start)
        b_start.pack(side='left', padx=5, pady=5)
        e_start = Entry(start_frame, width = 10)
        e_start.insert(0, f'{self.start}')
        e_start.pack(side='left', padx=5, pady=5)
        l_start = Label(start_frame, text=f"Start Date: {self.start}", font=("Arial Bold",15))
        l_start.pack(side='left', padx=5, pady=5)
        b_all_dates = Button(start_frame, text = "All Dates", command = all_dates)
        b_all_dates.pack(anchor= E, padx=5, pady=5)

        def recent_date():
            self.start = self.recent
            l_start.configure(text = f"Start Date: {self.start}", font=("Arial Bold",15))
            self.end = self.recent
            l_end.configure(text = f"End Date:  {self.end}", font=("Arial Bold",15))
        def set_end():
            self.end = np.datetime64(e_end.get())
            l_end.configure(text = f"End Date:  {self.end}")
        b_end = Button(end_frame, text = "Set Date", command = set_end)
        b_end.pack(side='left', padx=5, pady=5)
        e_end = Entry(end_frame, width = 10)
        e_end.insert(0, f'{self.end}')
        e_end.pack(side='left', padx=5, pady=5)
        l_end = Label(end_frame, text=f"End Date:  {self.end}", font=("Arial Bold",15))
        l_end.pack(side='left', padx=5, pady=5)
        b_recent_date = Button(end_frame, text = "Most Recent Date Only", command = recent_date)
        b_recent_date.pack(anchor= E, padx=5, pady=5)
           
         
        def all_data():
            global truthy
            truthy = not truthy
            self.c_total_v.set(truthy)
            self.c_deaths_v.set(truthy)
            self.c_recover_v.set(truthy)
            self.c_active_v.set(truthy)
            self.c_total_permil_v.set(truthy)
            self.c_deaths_permil_v.set(truthy)
            self.c_recover_permil_v.set(truthy)
            self.c_active_permil_v.set(truthy)
            self.c_change_permil_v.set(truthy)
            self.c_mortality_v.set(truthy)
            
        b_all_data = Button(data1_frame, text = "Select/Unselect All Data Types", command = all_data)
        b_all_data.pack(padx=5, pady=5)
        
        l_total = Label(data3_frame, text="Totals", font=("Arial Bold",10))
        l_total.pack(padx=5, pady=5)
        c_total = Checkbutton(data3_frame, text = 'Total Cumulative Cases', variable = self.c_total_v)
        c_total.pack(anchor= W, padx=5, pady=5)
        c_deaths = Checkbutton(data3_frame, text = 'Total Deaths', variable = self.c_deaths_v)
        c_deaths.pack(anchor= W, padx=5, pady=5)
        c_recover = Checkbutton(data3_frame, text = 'Total Recoveries', variable = self.c_recover_v)
        c_recover.pack(anchor= W, padx=5, pady=5)
        c_active = Checkbutton(data3_frame, text = 'Total Active Cases', variable = self.c_active_v)
        c_active.pack(anchor= W, padx=5, pady=5)

        l_permil = Label(data4_frame, text="Per Million", font=("Arial Bold",10))
        l_permil.pack(padx=5, pady=5)
        c_total_permil = Checkbutton(data4_frame, text = 'Cumulative Cases per Million People', variable = self.c_total_permil_v)
        c_total_permil.pack(anchor= W, padx=5, pady=5)
        c_deaths_permil = Checkbutton(data4_frame, text = 'Deaths per Million People', variable = self.c_deaths_permil_v)
        c_deaths_permil.pack(anchor= W, padx=5, pady=5)
        c_recover_permil = Checkbutton(data4_frame, text = 'Recoveries per Million People', variable = self.c_recover_permil_v)
        c_recover_permil.pack(anchor= W, padx=5, pady=5)
        c_active_permil = Checkbutton(data4_frame, text = 'Active Cases per Million People', variable = self.c_active_permil_v)
        c_active_permil.pack(anchor= W, padx=5, pady=5)
        
        l_permil = Label(data6_frame, text="Change per Million", font=("Arial Bold",10))
        l_permil.pack(padx=5, pady=5)
        c_change_permil = Checkbutton(data6_frame, text = 'Change in Active Cases per Million People over 1 Week', variable = self.c_change_permil_v)
        c_change_permil.pack(anchor= W, padx=5, pady=5)
        
        l_permil = Label(data7_frame, text="Mortality", font=("Arial Bold",10))
        l_permil.pack(padx=5, pady=5)
        c_mortality = Checkbutton(data7_frame, text = 'Mortality Rates', variable = self.c_mortality_v)
        c_mortality.pack(anchor= W, padx=5, pady=5)
        
        

        l_location = Label(location2_frame, text="Currently Selected Locations", font=("Arial Bold",15))
        l_location.pack(padx=5, pady=5)
        b_select_usa = Button(location3_frame, text = "Select USA as Location", command = lambda : select_usa(self))
        b_select_usa.pack(side='left', padx=20, pady=5)
        b_select_states = Button(location3_frame, text = "Select States as Locations", command = lambda : select_states(self))
        b_select_states.pack(side='left', padx=20, pady=5)
        b_select_counties = Button(location3_frame, text = "Select Counties as Locations", command = lambda : select_counties(self))
        b_select_counties.pack(side='left', padx=20, pady=5)
        b_clear_locations = Button(location3_frame, text = "Clear All Locations", command = lambda : clear_locations(self))
        b_clear_locations.pack(side='left', padx=20, pady=5)
        self.l_locations = Label(location4_frame, text=f"{self.locations_disp()}", wraplength=700)
        self.l_locations.pack(padx=5, pady=5)

        

        menubar = Menu(root)
        root.config(menu=menubar)
        helpmenu = Menu(menubar)
        menubar.add_cascade(label="Help", menu=helpmenu)
        #helpmenu.add_command(label="About...", command=About)

        
    def locations_disp(self):
        display = []
        locations_copy = self.locations.copy()
        
        all_states = True
        for k, v in get_st_abr.items():
            if v in self.locations:
                continue
            else:
                all_states = False
        if all_states == True:
            for k, v in get_st_abr.items():
                del locations_copy[v]
            locations_copy["All States Added"] = ["All States Added", "All_States"]
        elif "All States Added" in locations_copy:
            del self.locations["All States Added"]
        
        # Counties for each state
        for k, v in self.mapchartdict.items():
            all_counties = True
            for location in v:
                if location in self.locations:
                    continue
                else:
                    all_counties = False
            if all_counties == True:
                for location in v:
                    if location in locations_copy:
                        del locations_copy[location]
                if f"All {get_st_full[k]} Counties Added" not in locations_copy:
                    locations_copy[f"All {get_st_full[k]} Counties Added"] = [f"All {get_st_full[k]} Counties Added", "All_St_Counties"]
            else:
                if f"All {get_st_full[k]} Counties Added" in locations_copy:
                    del locations_copy[f"All {get_st_full[k]} Counties Added"]
        
        # Every County in US
        all_counties = True
        countiesdict1 = dict(zip(df.MapChart, df.County))
        for k, v in countiesdict1.items():
            if k in self.locations:
                continue
            else:
                all_counties = False
        if all_counties == True:
            for k, v in countiesdict1.items():
                if k in locations_copy:
                    del locations_copy[k]
            tempdict = locations_copy.copy()
            for k, v in tempdict.items():
                if v[1] == 'All_St_Counties':
                    del locations_copy[k]
            if 'All Counties Added' not in locations_copy:
                locations_copy['All Counties Added'] = ["All Counties Added", "All_Counties"]
        else:
            if 'All Counties Added' in locations_copy:
                del locations_copy['All Counties Added']

        for k, v in locations_copy.items():
            display.append(v[0])
        return display
        
        
    def check_data(self, type1):
        fail = False
        total = self.c_total_v.get() + self.c_deaths_v.get() + self.c_recover_v.get() + self.c_active_v.get()
        permil = self.c_total_permil_v.get() + self.c_deaths_permil_v.get() + self.c_recover_permil_v.get() + self.c_active_permil_v.get()
        check_one = total + permil + self.c_change_permil_v.get() + self.c_mortality_v.get()
        cross_groups = False
        if total != 0:
            if permil != 0 or self.c_change_permil_v.get() != 0 or self.c_mortality_v.get() != 0 :
                cross_groups = True
        if permil != 0:
            if self.c_change_permil_v.get() != 0 or self.c_mortality_v.get() != 0 :
                cross_groups = True
        if self.c_change_permil_v.get() != 0 and self.c_mortality_v.get() != 0 :
                cross_groups = True
        
        if type1 not in ['state_map', 'county_map'] and len(self.locations) == 0:
                messagebox.showinfo('No Locations Checked', 'Please check at least one location.')
                fail = True
        if check_one == 0:
            messagebox.showinfo('No Data Type Selected', 'Please check at least one data type.')
            fail = True
        if type1 in ['state_map', 'county_map']:
            if self.start != self.end:
                messagebox.showinfo('More than one Date Selected', 'Please select only one date for maps.')
                fail = True
            if check_one > 1:
                messagebox.showinfo('More than one Data Type Checked', 'Please check only one data type for maps.')
                fail = True      
        if type1 in ['one_location']:
            if cross_groups != False:
                messagebox.showinfo('Data Types from Different Groups Checked', 'Please check only data types of a common group. E.g. Totals, Per Million, etc.')
                fail = True
            if len(self.locations) > 1:
                messagebox.showinfo('More than one Location Checked', 'Please check only one location.')
                fail = True  
        if type1 in ['many_locations']:
            if check_one > 1:
                messagebox.showinfo('More than one Data Type Checked', 'Please check only one data type when comparing different locations.')
                fail = True
            if len(self.locations) == 1:
                messagebox.showinfo('Only One Location Checked', 'Please check multiple locations.')
                fail = True
        if type1 in ['one_location', 'many_locations'] and self.start == self.end:
                messagebox.showinfo('Only one Date Selected', 'Please select more than one date for graphs.')
                fail = True
        if fail == False:
            if type1 == 'state_map':
                type2 = self.make_data()
                Map_Export_Menu(type2[0], self.start, 'State')
            if type1 == 'county_map':
                type2 = self.make_data()
                Map_Export_Menu(type2[0], self.start, 'County')
            if type1 == 'one_location' or type1 == 'many_locations':
                Graphs(self.start, self.end, self.locations, self.make_data(), type1)
            if type1 == 'spreadsheet':
                Spreadsheet(self.start, self.end, self.locations, self.make_data())
    
    
    def make_data(self):
        self.data = []
        if self.c_total_v.get() == 1: self.data.append(1)
        if self.c_deaths_v.get() == 1: self.data.append(2)
        if self.c_recover_v.get() == 1: self.data.append(3)
        if self.c_active_v.get() == 1: self.data.append(4)
        if self.c_total_permil_v.get() == 1: self.data.append(5)
        if self.c_deaths_permil_v.get() == 1: self.data.append(6)
        if self.c_recover_permil_v.get() == 1: self.data.append(7)
        if self.c_active_permil_v.get() == 1: self.data.append(8)
        if self.c_change_permil_v.get() == 1: self.data.append(9)
        if self.c_mortality_v.get() == 1: self.data.append(10)
        return self.data


    def master_exit(self):
        root.destroy()


    def download_master(self):
        url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'
        urllib.request.urlretrieve(url, 'us-counties.csv')
        self.l_status.configure(text='New Master csv Downloaded!')


if __name__ == "__main__":
    root = Tk()
    main = MainWindow()
    main.pack(side='top')
    root.mainloop()